### Correção das redações

Alguns dos textos do corpus de redações veio com alguns caracteres faltando ou com marcações que atrapalham
a avaliação automática. Por conta disso, foi necessário buscar uma forma de corrigir esses problemas. Vale ressaltar que as correções dos textos foram feitas utilizando o texto completo. Assim, situações onde alguma
palavra com uma letra faltando, como em "onstituição", a correção foi feita considerando o valor correto "constituição", pois a perda da letra inicial ocorreu em algum dos processos de conversão.

In [ ]:
import re
import json
import pickle
import requests
import itertools
import matplotlib
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

Para executar a avaliação gramatical será utilizado o CoGroo, com o dicionário para Português Brasileiro do corretor JSpell. Como o software original do CoGroo foi feito em Java, fizemos um processo semelhante ao utilizado no cogroo4py. 

In [ ]:
# Gera uma lista com todas as redações a partir do arquivo json contendo
# os arquivos normalizados


df_redacoes = pd.read_json("../ccscore/data/todas_redacoes_norm.json")
df_redacoes


In [ ]:

def display_row(df, idx_row):
    redacao = df.iloc[idx_row]
    titulo = redacao['title']
    score = redacao['score']
    if 'texto' in df.columns.tolist(): 
        original = redacao['texto']
    else:
        original = redacao['original']
    fixed = redacao['fixed']
    review = redacao['review']
    criteria = redacao['criteria']

    print("Notas: ",criteria)
    print("\n","%"*100,"\n")
    print("Original: ", original)
    print("\n","%"*100,"\n")
    print("Corrigido: ", fixed)
    print("\n","%"*100,"\n")
    print("Revisado: ", review)


In [ ]:
# Ajusta as coluna das notas, conventendo para float todos os valores

df_redacoes['score'] = df_redacoes['score'].apply(lambda x:str(x).strip().replace(',','.'))
df_redacoes['score'] = pd.to_numeric(df_redacoes['score'], downcast="float")


In [ ]:
df_redacoes['score'].hist()

In [ ]:
novas_colunas = ['Competência 1', 'Competência 2', 'Competência 3', 'Competência 4', 'Competência 5']

for nc in novas_colunas:
    df_redacoes[nc] = df_redacoes['criteria'].apply(lambda x:x[nc])
    df_redacoes[nc] = df_redacoes[nc].apply(lambda x:str(x).strip().replace(',','.'))
    df_redacoes[nc] = pd.to_numeric(df_redacoes[nc], downcast="float")

df_redacoes

In [ ]:
# num. de linhas e colunas dos subplots
nrow=5
ncol=1
# dataframe (ou colunas) para plotar
df_list = [df_redacoes[nc] for nc in novas_colunas]
#df_list = [df1 ,df2, df3, df4, df5, df6]
fig, axes = plt.subplots(nrow, ncol)
# plotagem
count=0

if ncol == 1:
    for r in range(nrow):                
        df_list[count].hist(ax=axes[r])        
else:
    for r in range(nrow):
        for c in range(ncol):
            if count < len(df_list):
                df_list[count].plot(ax=axes[r,c])        
            count=+1

In [ ]:
df_redacoes['Competência 4'].hist()

In [ ]:
TEXTO_ENTRE_PARENTESES = r'\(.*?\)'  
PARENTESES_EM_BRANCO = r'(\([ ]{0,} \))'
PALAVRA_COLADA_PARENTESES = r'[a-zA-Z0-9]\('


#### Busca trechos contendo parênteses vazios nos textos

In [ ]:
def conta_ocorrencias(texto, ocorrencia, regex=False):
    ocorrencias = []
    try:
        q = re.findall(ocorrencia, texto)
        if len(q) > 0:
            ocorrencias += (i, q)
    except Exception as e:
        print(f"Erro na linha {i}")
        print(str(e))
        
    return len(ocorrencias)


def conta_ocorrencias_df(df, coluna, ocorrencia,exibe=False):
    total_ocorrencias = 0
    for i,r in df.iterrows():    
        q = re.findall(ocorrencia, str(r[coluna]))
        if exibe and len(q) > 0:
            print(f"{(i, q)}")
            
        total_ocorrencias += len(q)   
    return total_ocorrencias



#### Corrige parenteses em branco

In [ ]:
conta_ocorrencias_df(df_redacoes, 'original', PARENTESES_EM_BRANCO)

In [ ]:
df_redacoes['original'] = df_redacoes['original'].apply(lambda x:str(x))
df_redacoes['texto'] = df_redacoes['original'].apply(lambda x:re.sub(PARENTESES_EM_BRANCO, '', x))

In [ ]:
conta_ocorrencias_df(df_redacoes, 'texto', PARENTESES_EM_BRANCO)

#### Corrige palavras coladas em parênteses

In [ ]:
conta_ocorrencias_df(df_redacoes, 'texto', PALAVRA_COLADA_PARENTESES, exibe=False)

In [ ]:
def subst_colados(texto):
    ocorrencias = re.findall(PALAVRA_COLADA_PARENTESES, texto)
    
    for elem_ocor in ocorrencias:
        texto = texto.replace(elem_ocor, f"{elem_ocor[0]} {elem_ocor[1]}" )
        
    return texto

In [ ]:
df_redacoes['texto'] = df_redacoes['texto'].apply(lambda x:subst_colados(x))

In [ ]:
conta_ocorrencias_df(df_redacoes, 'texto', PALAVRA_COLADA_PARENTESES, exibe=False)

In [ ]:
def exporta_redacoes(df, coluna):    
    try:
        with open("texto_redacoes_export.txt", 'w') as f:        
            for i,r in df.iterrows():
                f.write(r[coluna])
    except Exception as e:
        print(str(e))


In [ ]:
exporta_redacoes(df_redacoes, 'texto')

### Avaliação das palavras incompletas

In [ ]:
from cogroo_interface import Cogroo
cogroo = Cogroo.Instance()

def check_cogroo(texto):
    cogroo = Cogroo.Instance()
    
    return cogroo.grammar_check(texto)


Carrega as marcações de correções das redações as marcas estão colocadas entre parênteses, como
por exemplo "(Sem vírgula)". Contudo, algumas das marcações no texto não são correções, mas sim 
elementos do próprio texto, como "MEC (Ministério da Educação e Cultura)"

In [ ]:
todas_palavras = pickle.load(open('../dicionarios/palavras_unitex_pb.pickle', 'rb'))


In [ ]:
marks_correcao = []
with open('../dicionarios/padroes_correcao.txt') as f_correcoes:
    marks_correcao = f_correcoes.readlines()


In [ ]:

def busca_marks_correcao(texto, l_correcoes):
    return [ x for x in set(re.findall('\(.*?\)', texto)) if x in l_correcoes]


def exclui_marks_correcao(texto, l_correcoes_excluir):
    for mark in l_correcoes_excluir:
        texto = texto.replace(mark, '')        
    return texto


def get_trecho(t,inicio,fim):
    return t[inicio-20:fim+20]


def get_sentence(t,inicio,fim):
    '''
    Dada uma posição de inicio e fim dentro de um trecho de texto,
    busca a setença a qual esse treco faz parte. De forma simples,
    busca o proximo ponto (.) precedido de espaço.
    '''
    # Posicao do proximo ponto após o intervalo
    pos_ponto_fim = t.find('. ', fim, len(t))
    if pos_ponto_fim == -1:
        pos_ponto_fim = t.find('.', fim, len(t))
        pos_ponto_fim += 1
    else:        
        pos_ponto_fim += 2
    
    # Reverte a string e pega o primeiro " ." na string invertida
    # que é equivalente ao primeiro " ." anterior ao intervalo
    t_rev = t[::-1]
    ponto_busca = len(t) - fim+(fim-inicio)
    t_rev = t_rev[ponto_busca:]
    pos_ponto_ini = t_rev.find(' .')
    
    # Retorna a sentença contida entre os pontos imediatamente anterior
    # e posterior ao intervalo
    return(t[inicio-pos_ponto_ini : pos_ponto_fim])


def destaca_texto(texto, destaque):
    pos_ini = texto.find(destaque)
    pos_fim = pos_ini + len(destaque)
    
    return f"{texto[:pos_ini]}[[{destaque}]]{texto[pos_fim:]}"


In [ ]:
token_descarte = ["prop", ",", "num", "pron-indp", "--", ".", "\"", "..."]
def busca_palavras_erradas(check, dicionario, analisa=False):
    token_descarte = ["prop", ",", "num", "pron-indp", "--", ".", "\"", "..."]
    tokens_outdict = []
    
    if analisa:
        check = check_cogroo(check)
    
    for s in check.sentences:  
        for token in s.tokens:
            str_t = str(token.lexeme)
            if token.pos not in token_descarte and \
               len(str_t) > 1 and \
               "_" not in token.lexeme:
                if token.lemma == "" and token.lexeme.lower() not in dicionario:                                        
                    if not token.lexeme.startswith('-'):                        
                        tokens_outdict.append( (str(token.lexeme), token.start, token.end) )
    
    return tokens_outdict


In [ ]:
# Essa execucação é bem demorada. Lembrar se serializar o resultado, para não
# ter que executar seguidas vezes

texto_analisado = {}
erros_por_redacao = []
erros = []
with tqdm(total=df_redacoes.shape[0]) as pbar:    
    for i_redacao, r in df_redacoes.iterrows():
        pbar.update(1)
        
        '''
        if i_redacao < 100:
            continue
        if i_redacao > 130:
            break
        '''
        
        try:
            texto = check_cogroo(r['texto'])
            texto_analisado[i_redacao] = texto
            erros_por_redacao.append(len(busca_palavras_erradas(texto)))
        except Exception as e:
            erros_por_redacao.append(0)
            texto_analisado[i_redacao] = None            
            erros.append( (i_redacao,str(e)) )


In [ ]:
# Serialização do resultado das analises do Cogroo sobre os textos
#pickle.dump(texto_analisado, open("docs_textos_analisados.pickle", 'wb'))

In [ ]:
# Separa os numeros das redações que apresentaram erro
redacao_com_erro,_ = zip(*erros)
redacao_com_erro

# Atribui zero para coluna "erros" das 
# redações que apresentaram erros na analise

for num in redacao_com_erro:
    df_redacoes.at[num, 'palavras desconhecidas'] = 0

df_redacoes[df_redacoes.index.isin(redacao_com_erro)]

df_redacoes

In [ ]:
# Prepara uma lista contendo a qtde de erros por redação
# para posterior inserção no DafaFrame

num_erros = []
i = 0
for idx_redacao,r in df_redacoes.iterrows():    
    if r['palavras desconhecidas'] != 0:
        num_erros.append(erros_por_redacao[i])
        i += 1
    else:
        num_erros.append(0)
        

len(num_erros) == df_redacoes.shape[0]


In [ ]:
# Insere a lista de erros na coluna do DataFrame
#df_redacoes['palavras desconhecidas'] = num_erros

In [ ]:
# Exportação do DataFrame de redações, com os ajustes
#df_redacoes.to_json('redacoes_correcao_parcial.json', orient='records')

# Serialização do DataFrame de redações, com os ajustes
#pickle.dump(df_redacoes, open('dataframe_redacoes_correcao.pickle', 'wb'))

#df_redacoes = pickle.load(open('dataframe_redacoes_correcao.pickle','rb'))

In [ ]:
df_redacoes[df_redacoes['score'] < 11]
total = []
criterio1 = []
criterio2 = []
criterio3 = []
criterio4 = []
criterio5 = []

for idx_redacao, redacao in df_redacoes.iterrows():
    nota = float(redacao['score'])
    n1 = float(redacao['Competência 1'])
    n2 = float(redacao['Competência 2'])
    n3 = float(redacao['Competência 3'])
    n4 = float(redacao['Competência 4'])
    n5 = float(redacao['Competência 5'])
    
    if nota < 11:
        nota *= 100
        n1 *= 100
        n2 *= 100
        n3 *= 100
        n4 *= 100
        n5 *= 100
        
    total.append(nota)
    criterio1.append(n1)
    criterio2.append(n2)
    criterio3.append(n3)
    criterio4.append(n4)
    criterio5.append(n5)
    


In [ ]:
 df_redacoes[(df_redacoes['score'] == 0)]

In [ ]:
df_redacoes['score'] = total
df_redacoes['Competência 1'] = criterio1
df_redacoes['Competência 2'] = criterio2
df_redacoes['Competência 3'] = criterio3
df_redacoes['Competência 4'] = criterio4
df_redacoes['Competência 5'] = criterio5

In [ ]:
df_redacoes.columns = ['title', 'url', 'score', 'original', 'fixed', 'review', 'comments',
       'errors', 'criteria', 'Competência 1', 'Competência 2', 'Competência 3',
       'Competência 4', 'Competência 5', 'texto', 'palavras desconhecidas']

In [ ]:
df_redacoes['idx original'] = df_redacoes.index.values.tolist() 

In [ ]:
df_redacoes

In [ ]:
df_redacoes['qtd palavras'] = df_redacoes['texto'].apply(lambda x:len(re.findall(r'\S+', x))).to_list() 

In [ ]:
df_1_erro = pd.DataFrame(df_redacoes[(df_redacoes['palavras desconhecidas'] < 3) & (df_redacoes['texto'] != "")])

In [ ]:
df_1_erro[['score']].hist()

In [ ]:
df_selected = pd.DataFrame(df_selected[(df_selected['qtd palavras'] > 250) & 
                                       (df_selected['palavras desconhecidas'] < 5) &
                                       (df_selected['texto'] != "") ])

In [ ]:
# Exportação do DataFrame de redações, com os ajustes
#df_selected.to_json('redacoes_selected_parcial.json', orient='records')

In [ ]:
# Serialização do DataFrame de redações, com os ajustes
#pickle.dump(df_selected, open('dataframe_selected.pickle', 'wb'))
df_selected = pickle.load(open('dataframe_selected.pickle','rb'))

In [ ]:
#df_selected.index[0]
df_selected

In [ ]:
df_selected.reset_index(drop=True, inplace=True)

In [ ]:
df_selected

In [ ]:
# Exclui redações que não estão na lista de anotações do PALAVRAS
#excluir_idx_original = [8695, 8747, 8787, 8807, 10874, 10926, 10986]
#excluir_idx = [i for i,x in df_selected.iterrows() if x['idx original'] in excluir_idx_original]
#excluir_idx

In [ ]:
#df_selected.drop(excluir_idx, inplace = True)

In [ ]:
df_selected.iloc[3814]

In [ ]:
display_row(df_selected, 1710)

### Alinha as redações no DataFrame com os objetos de análise do Cogroo

In [ ]:
texto_analisado = pickle.load(open("docs_textos_analisados.pickle", 'rb'))

In [ ]:
analises_cogroo = {}
for i,r in df_selected.iterrows():
    idx_original = int(r['idx original'])
    if idx_original in texto_analisado.keys():
        analises_cogroo[i] = texto_analisado[idx_original]
    else:
        print(f"Analise idx {idx_original} não encontrada!")

In [ ]:
print(analises_cogroo[0])
df_selected.iloc[0]['texto']

### Alinha as redações no DataFrame com os textos anotados pelo PALAVRAS

In [ ]:
todas_analises_palavras = {}
redacao = None
idx_redacao = 0
with open("redacoes/all_texts_annotated_palavras.txt", 'rb') as f_palavras:
    for linha in f_palavras:
        l = linha.decode("utf8","ignore")
        if "ARQUIVO: \"redacoes/TXT/r_" in l:  # novo texto                                    
            if not(redacao is None):                    
                todas_analises_palavras[idx_redacao] = "".join(redacao)
                idx_redacao = int(l.split('_')[1])
                #idx_redacao += 1
            else:
                idx_redacao = int(l.split('_')[1])
                
            redacao = []
        elif "s_id=" in l or "</s>" in l:
            continue
        else:            
            redacao.append(l)
        
    todas_analises_palavras[idx_redacao] = "".join(redacao)


In [ ]:
len(todas_analises_palavras)

In [ ]:
i_pal = 0
analises_palavras = {}
redacoes_excluir = []
for i,r in df_selected.iterrows():
    idx_original = int(r['idx original'])
    if idx_original in todas_analises_palavras.keys():
        analises_palavras[i] = todas_analises_palavras[idx_original]
    else:
        print(f"Não foi encontrada anotação para a redação: {i} - idx_original: {idx_original}")
        analises_palavras[i] = None
        redacoes_excluir.append(i)
    

In [ ]:
#len(analises_palavras) == len(df_selected)
#df_selected.iloc[8695]['idx original']
redacoes_excluir

In [ ]:
analises_palavras[3814]

In [ ]:
analises_cogroo[3814]

In [ ]:
len(analises_palavras) == len(analises_cogroo)
#len(analises_cogroo)

In [ ]:
df_selected_ok = df_selected.drop(redacoes_excluir)
df_selected_ok.reset_index(drop=True, inplace=True)

In [ ]:
analises_palavras_ok = {}
analises_cogroo_ok = {}
i = 0
for pos in range(df_selected.shape[0]):
    if pos not in redacoes_excluir:
        analises_palavras_ok[i] = analises_palavras[pos]
        analises_cogroo_ok[i] = analises_cogroo[pos]
        i+=1
    del analises_palavras[pos]
    del analises_cogroo[pos]
    
        

In [ ]:
print(len(analises_palavras_ok) == len(analises_cogroo_ok))
print(len(analises_palavras_ok) == df_selected_ok.shape[0])

In [ ]:
print(analises_palavras_ok[3814])
print(analises_cogroo_ok[3814])
print(df_selected_ok.iloc[3814]['texto'])

In [ ]:
r = []
erros = []
#p1_an = ""
#p1_sel = ""
for i in range(df_selected_ok.shape[0]):
    p1_an = ""
    p1_sel = ""
    try:
        an_p = analises_palavras_ok[i]
        p1_an = an_p.split()[0]

        sel_t = df_selected_ok.iloc[i]['texto']
        p1_sel = sel_t.split()[0]
        
        r.append(p1_an in p1_sel)
    except Exception as e:
        erros.append((i,f"p1_an: {p1_an}",f"p1_sel: {p1_sel}",str(e)))
    

    


In [ ]:
erros
#[3814, 3847, 3867, 3877, 4769, 4802, 4829]

In [ ]:
an_p = analises_palavras_ok[8]
p1_an = an_p.split()[0]

sel_t = df_selected_ok.iloc[8]['texto']
p1_sel = sel_t.split()[0]

print(f"p1_an: {p1_an}",f"p1_sel: {p1_sel}")
p1_an in p1_sel

In [ ]:
from collections import Counter
Counter(r)

[i for i,x in enumerate(r) if not x]

In [ ]:
# Serializa as versões finais do dataframe contendo a lista de todas as redações
# o dicionário com as analises do Cogroo e o dicionário com as analises do 
# PALAVRAS. Cada item nos dicionarios esta indexado pela posição no DataFrame

pickle.dump(df_selected_ok, open("Corpus_Redacoes_DATAFRAME.pickle", 'wb'))
pickle.dump(analises_palavras_ok, open("Corpus_Redacoes_PALAVRAS.pickle", 'wb'))
pickle.dump(analises_cogroo_ok, open("Corpus_Redacoes_COGROO.pickle", 'wb'))


In [ ]:
df_selected_ok.to_json('Corpus_Redacoes_DATAFRAME.json', orient='records')

In [ ]:
token_descarte = ["prop", ",", "num", "pron-indp", "--", ".", "\"", "..."]
erros_redacoes = set([])
tokens_outdict = set([])
redacoes_com_erros = set([])

for i_redacao, r in df_redacoes.iterrows():
    if i_redacao > 100: 
        break
    
    check = check_cogroo(r['texto'])
        
    for erro in check.mistakes:
        texto    = erro.context 
        id_erro  = erro.rule_id
        msg_erro = erro.short_msg
        inicio   = erro.start
        fim      = erro.end    
        trecho_erro = texto[inicio:fim]
        #print("Trecho Erro", trecho_erro)
        #sentenca_erro = get_sentence(texto,inicio,fim)        
        erros_redacoes.add(f"{id_erro} - {msg_erro}")
        #print(f"Erro: {id_erro} - {msg_erro} \n")
        #print(f"\tSentença: {sentenca_erro}")
        #print(f"\t{destaca_texto(sentenca_erro,trecho_erro)} \n\n")
        redacoes_com_erros.add(i_redacao)
    for s in check.sentences:        
        for token in s.tokens:
            str_t = str(token.lexeme)
            if token.pos not in token_descarte and \
               len(str_t) > 1 and \
               "_" not in token.lexeme:
                if token.lemma == "" and token.lexeme.lower() not in todas_palavras:                    
                    #print(f"Token <{str(token),str(token.pos)}> não encontrado no dicionário!")                    
                    if not token.lexeme.startswith('-'):
                        #tokens_outdict.add(f"{str(token.lexeme)}<->{str(token.pos)}")
                        tokens_outdict.add(f"Redação: {i_redacao} {str(token.lexeme)}")
       

In [ ]:
df_selected[(df_selected['qtd palavras'] > 250) & 
            (df_selected['palavras desconhecidas'] < 5) &  
            (df_selected['Competência 4'] >= 150) &
            (df_selected['score'] < 500)]

In [ ]:
with open('all_fixed.txt', 'w') as f:
    f.write(all_fixed)

In [ ]:
marcacoes_excluir = ["(Vírgula)", "~~", "(Por quê?)"]
texto = fixed
for x in marcacoes_excluir:
    texto = texto.replace(x, "")

print(texto)

In [ ]:
import re
marks_correcao = set(re.findall('\(.*?\)', all_fixed))
marks_correcao = sorted(marks_correcao)
print("\n".join(marks_correcao))

In [ ]:
correcoes_revisao = [x for x in marks_correcao if any(y == x for y in padroes_correcoes)]
correcoes_revisao

In [ ]:
padroes_mark_buscar = []
marks_excluir = []

In [ ]:
checks_redacoes = []

for r in redacoes:
    checks_redacoes.append(cogroo.grammar_check(r))
    

In [ ]:
len(checks_redacoes)

In [ ]:
erros_redacoes

erros_descarte = ["punctuation:BEFORE_SENTENCES", "'punctuation:EXTRA_PUNCTUATION",
                 "repetition:DUPLICATED_TOKEN", "space:EXTRA_AFTER_LEFT_PUNCT",
                 "space:EXTRA_BEFORE_RIGHT_PUNCT", "space:EXTRA_BETWEEN_WORDS", 
                 "space:MISSING_SPACE_AFTER_PUNCT",
                 "xml:1", "xml:2", "xml:3", "xml:4", "xml:5", "xml:6", "xml:87"]


In [ ]:
tokens_outdict
#sorted_outdict = sorted(tokens_outdict, key=lambda x:x.split("-")[1].strip('|'))
sorted_outdict = sorted(tokens_outdict, key=lambda x:x[0])
print("\n".join(sorted_outdict))


In [ ]:
import sys
sys.path.insert(0, "../dicionarios/")
from corretor_palavras import Corretor

corretor = Corretor("../dicionarios/frequencia_palavras_pt_br.txt")
for palavra in sorted_outdict:
    w = corretor.sugestao_correcao(palavra)                
    if len(w) > len(palavra):
        print('correção({}) => {} ({})'.format(palavra, w, corretor.WORDS[w] if w in corretor.WORDS else 0))

In [ ]:
len(corretor.WORDS)

In [ ]:
len(redacoes_com_erros)

In [ ]:
redacoes[0]

In [ ]:
#print("\n".join([str(x) for x in redacoes if "onstitucional" in x and "constitucional" not in x]))
[str(x) for x in redacoes if "rasileira(" in x]

In [ ]:
c = cogroo.grammar_check("O home fez o seu cumprimento e foi embora de carrrroioeioa.")

#c = cogroo.analyze("caroroiro")  

In [ ]:
for s in c.sentences:
    for token in s.tokens:
        print(str(token), token.lexeme, token.lemmas)

In [ ]:
id_erro = checks_redacoes[1].mistakes[0].rule_id
msg_erro = checks_redacoes[1].mistakes[0].short_msg
inicio = checks_redacoes[1].mistakes[0].start
fim = checks_redacoes[1].mistakes[0].end
texto = checks_redacoes[1].text 

print(f"Erro: {id_erro} - {msg_erro} - {texto[inicio:fim]}")

In [ ]:
padroes_correcoes = set("""( "por isso" entre vírgulas)
( acento agudo)
( o "os" soa melhor e pode substituir o verbo possuem)
((Sem vírgula)
((Vírgula)
(***)
(, )
(, de acordo)
(, em)
(, entre outras)
(, já)
(, o que implica)
(, o que)
(, por exemplo)
(, uma vez que)
(,)
(,como dizia)
(,entregues pelo mordomo pessoal,)
(,que)
(-)
(-os)
(-se)
(. ponto final)
(.)
(...)
(.ponto final)
(.ponto final, início com letra maiúscula – Muitos)
(:)
(;)
(?)
(??)
(???)
(A campanha)
(A experiência do)
(A frase)
(A guerra invisível)
(A internet é)
(A notícia de)
(A partir)
(A reforma aumentará a carga horária. Observe que existe um equívoco nessa afirmação)
(A reforma)
(A – início de frase com letra maiúscula)
(Afeta de que maneira?)
(Afetar em que sentido?)
(Afetaria de que maneira?)
(Afinal,)
(Afirmação equivocada)
(Agora, frente à)
(Ainda)
(Algumas)
(Alguns deles quem?)
(Alguns)
(Além)
(Ambos)
(Amenizadas de que maneira?)
(Ao)
(Apenas punir)
(Apresenta ideais confusas)
(Apresenta o poblema da infraestrutura, mas não articula para uma solução)
(Apresente esse fato)
(Apresente esse índice para sustentar o seu argumento)
(Apresente fatos para afirmar essa colocação)
(Apresente fatos que comprovem os acontecimentos que você citou)
(Apresente pontos em comum)
(Apresente um fato que evidencie o porquê disso estar acontecendo)
(Assim)
(Até)
(Boa argumentação, mas precisa ser desenvolvida com mais clareza)
(,)
(C )
(Cite alguns exemplos)
(Cite alguns)
(Cite esses países no seu texto)
(Cite exemplos do que seria ideal)
(Cite exemplos)
(Cite o nome)
(Cite uma fonte)
(Cite)
(Cite-as)
(Cite-os)
(Clichê)
(Coloque entre aspas)
(Com – letra maiúscula)
(Comece falando em que momento histórico isso aconteceu)
(Como a reforma poderia tirar o conhecimento do aluno? Desenvolva melhor essa ideia)
(Como assim? A ideia está confusa)
(Como assim? Apresente evidencias que possam comprovar isso)
(Como assim? Apresente exemplos acerca do fato para dar validade a sua afirmação)
(Como assim? As ideias aqui presentes estão confusas. Cuidado ao fazer algumas afirmações. Reflita)
(Como assim? Cuidado ao fazer algumas afirmações)
(Como assim? Cuidado ao fazer esse tipo de afirmação)
(Como assim? Cuidado ao fazer essee tipo de colocação. Realmente é possível dizer que uma pessoa é mais inteligente que a outra? Reflita)
(Como assim? Desenvolva essa ideia com mais clareza)
(Como assim? Desenvolva essa ideia no texto)
(Como assim? Desenvolva essa ideia)
(Como assim? Desenvolva melhor essa ideia)
(Como assim? Desenvolva melhor essa ideia, pois ela está confusa)
(Como assim? Discuta no texto)
(Como assim? Essa afirmação ficou estranha. Reflita)
(Como assim? Essa ideia está confusa. Reformule)
(Como assim? Essa ideia não está clara)
(Como assim? Estamos no século XXI)
(Como assim? Evidencie no texto)
(Como assim? Ficou confuso)
(Como assim? Não ficou claro o que você queria dizer)
(Como assim? Não ficou claro)
(Como assim? Observe e reflita sobre essa afirmação)
(Como assim? Observe que há um equívoco quando você afirma isso, visto que o governo não ignora o Enem)
(Como assim? Observe que há um erro na construção de sentido. Reformule essa ideia)
(Como assim? Problema na construção da argumentação)
(Como assim? Quais?)
(Como assim? Reelabore essa afirrmação)
(Como assim? Reflita acerca dessa afirmação)
(Como assim? Reflita e reelabore essa afirmação, pois ela está confusa)
(Como assim? Reflita sobre essa informação. De fato, isso não acontece?)
(Como assim? Reformule essa afirmação)
(Como assim? Reformule essa ideia)
(Como assim? Reformule essa ideia. Qual seria o problema em questão? Reflita)
(Como assim? Reformule)
(Como assim? Reformule, pois está confuso)
(Como assim? Reformule, tendo em vista que é contraditório de acordo com o que você menciona no período anterior)
(Como assim?)
(Como asssim? Cuidado ao afirmar isso. Reformule essa ideia)
(Como asssim? Reformule essa ideia)
(Como isso vai resolver o problema? Deixe evidente)
(Como no)
(Como seriam as mudanças bem formuladas? Fale sobre isso no seu texto)
(Como)
(Como? Apresente no texto)
(Confuso)
(Conscientizar de que maneira? Apresente isso no texto)
(Conscientizar de que maneira?)
(Consuela)
(Cuidado ao afirmar isso)
(Cuidado ao afirmar isso, pois pode ferir os direitos humanos. Punir em que sentido?)
(Cuidado ao afirmar isso, visto que a reforma entrará em vigor em 2018)
(Cuidado ao afirmar isso, visto que vai de encontro aos direitos humanos)
(Cuidado ao afirmar isso. Reformule essa ideia)
(Cuidado ao fazer algumas afirmações)
(Cuidado ao fazer certas afirmações)
(Cuidado ao fazer essa afirmação)
(Cuidado ao fazer essa afirmação. Apresente fatos que encaminhem as discussões acerca do tema)
(Cuidado ao fazer essa afirmação. Reformule essa ideia)
(Cuidado ao fazer esse tipo de afirmação)
(Cuidado ao fazer generaliações)
(Cuidado ao fazer generalizações)
(Cuidado ao generalizar)
(Cuidado ao generalizar. Reflita acerca dessa informação)
(Cuidado ao propor isso. Que tipo de punição severa?)
(Cuidado com algumas afirmações)
(Cuidado com algumas afirmações. Reveja essa ideia e a reelabore)
(Cuidado com esse termo "invadir")
(Cuidado com esse tipo afirmação. Lembre-se dos Direitos Humanos)
(Cuidado com esse tipo de afirmação)
(De certa forma, o apelido também pode ser uma forma de preconceito. Reflita)
(De que forma isso acontece?)
(De que maneira isso seria possível)
(De que maneira? Deixe esse posicionamento evidente em seu texto)
(De que maneira? Discuta isso no texto. Desenvolva essa ideia)
(De que maneira? Dê um exemplo do que pode ser feito para reverter essa situação)
(De que maneira?)
(De que modo isso seria possível? Apresente isso no texto)
(De que modo)
(De que modo?)
(Deixe evidente o que seria essa reforma geral)
(Deixe o motivo evidente no seu texto)
(Depois de ponto final inicia-se o período com letra maiúscula)
(Descreva essa situação em seu texto)
(Descreva o que seria esse cenário triste do ensino médio)
(Desenvola essa ideia, pois ela está limitada)
(Desenvolva as ideias desse parágrafo)
(Desenvolva essa afirmação de forma mais clara)
(Desenvolva essa afirmação)
(Desenvolva essa ideia com mais clareza. Deixe evidente no texto)
(Desenvolva essa ideia de forma mais clara)
(Desenvolva essa ideia de modo mais claro)
(Desenvolva essa ideia e apresente exemplos que justifiquem o porquê disso)
(Desenvolva essa ideia e apresente exemplos)
(Desenvolva essa ideia e exemplifique)
(Desenvolva essa ideia no seu texto para deixar a sua argumentação clara e apresente exemplos)
(Desenvolva essa ideia no texto)
(Desenvolva essa ideia para que fique claro o seu posicionamento)
(Desenvolva essa ideia)
(Desenvolva essa ideia, pois ela está confusa)
(Desenvolva essa ideia, pois ela não está clara)
(Desenvolva essa ideia; explique o porquê)
(Desenvolva mais essa ideia)
(Desenvolva melhor essa ideia)
(Desenvolva)
(Desse modo)
(Deste modo)
(Deve ainda)
(Deveria ou deve)
(Deveria ou deve? Reflita)
(Dia – inicie com letra maiúscula)
(Diante d)
(Diante de)
(Diferente como? Deixe evidente no texto)
(Discussão limitada)
(Discuta essa ideia no texto)
(Discutida por quem? Exemplifique)
(Dispensável, pois ela está no mesmo campo semântico de humilhação e opressão)
(Dois pontos)
(Duas conjunções que expressam a ideia de conclusão)
(Dê exemplo)
(Dê exemplos de mudanças)
(Dê exemplos dessas agressões)
(Dê exemplos dessas situações)
(Dê exemplos desse tipo de agressão)
(Dê exemplos desses atos violentos)
(Dê exemplos para reafirmar esse posicionamento)
(Dê exemplos)
(Dê exemplos. Desenvolva mais essa ideia nesse parágrafo de desenvolvimento)
(Dê um exemplo disso)
(Dê um exemplo)
(E o que isso pode acarretar? Discuta no texto)
(Ela)
(Elas não estão preparadas de que maneira?Fale sobre isso no seu texto)
(Ele, – com letra maiúscula e vírgula)
(Eles não foram tão discutidos no seu texto. Dê exemplos)
(Eliminar de que maneira? Cuidado ao afirmar isso, pois vai de encontro aos direitos humanos)
(Em algum momento isso foi possível? Reflita acerca dessa afirmação)
(Em relação à)
(Em)
(Entre aspas pois é uma citação)
(Entre vírgulas)
(Envidencie em seu texto o que há de ruim)
(Essa ideia não está tão evidente no seu texto)
(Essa ideia não está tão evidente no seu texto. Reformule a sua linha de argumentação para que isso fique claro)
(Essa ideia não está tão evidente no texto)
(Essas são)
(Esse)
(Esses)
(Esta renúncia)
(Esta)
(Este é)
(Evidencie isso por meio de fatos)
(Evite esse tipo de construção no texto)
(Evite esse tipo de expressão clichê no texto)
(Evite o piegas no texto)
(Exemplifique e discuta as causas)
(Exemplifique)
(Exemplifique; ponto final)
(Explique a importância dessas disciplinas no seu texto e que disciplinas são essas)
(Explique a razão disso)
(Explique como o bullying se desenvolve no âmbito sexual)
(Explique melhor essa ideia)
(Explique meslhor essa afirmação)
(Explique o porquê de tal afirmação)
(Explique o porquê de tanta repercussão)
(Explique o porquê de ter dado certo)
(Explique o porquê dessa afirmação)
(Explique o porquê dessa afirmação. Exemplifique)
(Explique o porquê dessa reforma ser mal elaborada)
(Explique o porquê dessas matérias serem essenciais para o conhecimento de mundo)
(Explique o porquê dessas mudanças dividirem opiniões)
(Explique o porquê diss pois essa ideia não está clara em seu texto)
(Explique o porquê disso estar acontecendo)
(Explique o porquê disso estar acontecendo. Evidencie isso no seu texto)
(Explique o porquê disso ocorrer)
(Explique o porquê disso)
(Explique o porquê)
(Explique o que seria uma reforma bem elaborada)
(Explique que reforma seria essa)
(Expressão clichê)
(Expressão clichê. Sem vírgula)
(FEBEM’s – diga o que significa a sigla)
(Facilitado de que maneira? Cuidado ao fazer certas generalizações. O conhecimento está, de fato, facilitado a todos? Reflita)
(Fale mais sobre essa ideia no seu texto; exemplique)
(Fale mais sobre essa planificação escolar)
(Fale mais sobre essa situação crítica)
(Fale mais sobre esse acordo)
(Fale mais sobre isso no seu texto)
(Fale mais sobre isso;)
(Fale mais sobre o que seria essa reforma no seu texto)
(Fale sobre a atual situação da educação brasileira no seu texto)
(Fale sobre a estrutura que você considera ideal)
(Fale sobre a proposta)
(Fale sobre essas mudanças)
(Fale sobre essas rupturas no seu texto)
(Fale sobre isso no seu texto)
(Fale sobre o porquê dessa discussão)
(Fale um pouco mais desses padrões)
(Falhar em que sentido?)
(Falou o quê?)
(Falta de conhecimento sobre o quê?)
(Famosos)
(Fica muito vago. Cite ao menos um exemplo)
(Formule melhor as ideias desse período)
(Formule melhor as ideias deste período)
(Houve um)
(Houve)
(Há no Brasil)
(Há também)
(Há uma lei que trate disso)
(Há)
(Ideia clichê)
(Ideia que precisa ser desenvolvida com mais clareza)
(Ideias confusas deste período.Revise-o)
(Ideias confusas)
(Ideias confusas. Elabore-as com mais clareza. Exemplifique mais)
(Incentivar de que maneira?)
(Informação dispensável)
(Iniciação de paragráfo sempre o uso de letra maiúscula)
(Iniciação de paragráfo)
(Intimidar de que maneira?)
(Isso acontece só com os alunos? Reflita e reformule essa colocação)
(Isso acontece só no Brasil? Reflita acerca dessa informação)
(Isso não está claro no seu texto)
(Isso não está explícito no texto)
(Isso não está tão claro no texto)
(Isso não está tão evidente no texto)
(Isso não fica evidente no texto. Reelabore a sua argumentação expondo fatos)
(Isso não ocorrerá em todos os casos)
(Isso o quê?)
(Isso pode ser feito)
(Isso é reflexo da)
(Isso)
(Isso, de fato, é tão simples assim? Reflita)
(Letra maiúscula em início de frase)
(Letra maiúscula)
(Letra maiúsula)
(Letra minúscula)
(Marca de oralidade)
(Marcas de oralidade)
(Mas isso decorre de um problema psiicológico. Reflita)
(Mas isso está previsto na reforma)
(Mas o “por que”)
(Mas)
(Melhorar sob qual perspectiva? Exemplifique)
(Melhoraria de que forma? Procure evidenciar isso)
(Muda de que maneira? Deixe isso claro no texto)
(Muitas das)
(Muito se)
(Muitos quem? Por quê?)
(Muitos quem?)
(Muitos)
(No Brasil)
(No Brasil,muitos jovens são)
(No dia)
(No que isso influência na nova reforma?)
(No)
(Nos)
(Notado de que maneira? Deixe isso claro no seu texto. Exemplifique)
(Não compreende a estrutura do texto dissertativo-argumentativo)
(Não está evidente no seu texto)
(Não está explícito no texto)
(Não ficou claro. Refaça esse período)
(Não há "deste")
(Não use primeira pessoa no texto dissertativo-argumentativo)
(Não é necessário)
(Não é tão comum assim. Reflita acerca dessa afirmação)
(Não – com letra maiúscula)
(O que ainda falta ser discutido? Deixe isso claro notexto)
(O que de fato seria o diferente? Discuta no texto)
(O que ele afirmava? Abra aspas e cite o ponto de vista dele)
(O que está ruim? Deixe isso evidente em seu texto)
(O que poderia sanar os problemas? Deixe isso evidente no seu texto)
(O que seria ensino clássico e contemporâneo? Fale sobre isso no seu texto)
(O que seria essa "educação de qualidade?)
(O que seria essa verdadeira mudança?)
(O que seria um comportamento "fora do normal"?)
(O que seria uma conversa relevante?)
(O que seria uma educação mais igualitária? Evidencie isso no seu texto)
(O que seria uma sociedade evoluída?)
(O que seriam melhores condições de trabalho? Exemplifique)
(O que são coisas?)
(O que você quis dizer?)
(O que vocês quis dizer aqui? Ficou confuso)
(O que é coisa?)
(O quê? Evidencie isso no seu texto)
(O quê?)
(O vírus foi)
(O – maiúsculo)
(Observando)
(Observe a falta de coesão e coerência)
(Observe essa afirmação e procure relacionar a realidade do nosso país com a realidade social da Finlância)
(Observe essa afirmação e reflita. Não ficou estranho? )
(Observe essa afirmação)
(Observe o uso de letra maiúscula após a vírgula)
(Observe que essa informação não apresenta coerência com o que foi discutido anteriormente)
(Observe que há problema na construção de sentido. Reformule)
(Observe que há problemas na construção de sentido. Reformule)
(Observe que há um equívoco nessa afirmação)
(Observe que há uma diferença muito grande entre escola pública e escola particular)
(Observe que há uma generalização aqui, pois nem todos fazer curso técnico após terminar o ensino médio)
(Observe que não há um direcionamento de quando é o fim da citação. Reveja isso)
(Observe que o Enem foi feito pelos alunos um mês após a data marcada para o acontecimento do exame)
(Observe que o MEC e o Ministério da educação são a mesma coisa, o que muda é a abreviação)
(Observe que todos os ambientes citados por você fazem parte de um meio social. Logo, essa colocação é dispensável)
(Onde essa informação foi apresentada? A evidencie no texto)
(Onde isso ocorreu? Dê exemplos)
(Os atos verbais não são violentos? Relflita e reformule, pois ficou confuso)
(Os dois termos apresentam a mesma ideia)
(Os outros quem?)
(Outro)
(Outros quem?)
(Outros)
(Para que fim? Discuta isso no seu texto)
(Paragráfo)
(Perseguir o quê?)
(Período com ideias confusas.Revise)
(Período confuso)
(Período muito extenso)
(Pesquisa realizada por quem?)
(Piorando em que sentido? Deixe isso evidente no seu texto)
(Pleonasmo)
(Poblema na construção de sentido desse período. Reformule)
(Poderia – introduzir com letra maiúscula)
(Ponto fina)
(Ponto final para iniciar um novo período)
(Ponto final para iniciar um novo período; falta de coesão)
(Ponto final)
(Ponto final. Aperfeiçoar de que maneira?)
(Ponto fnal)
(Por que ela pode ser relacionada ao Fordismo? Deixe isso evidente no texto)
(Por que ela é bem-vinda? Apresente no texto)
(Por que ele é defasado? Explique)
(Por que eles serão afetados? Explique)
(Por que esses países são exemplos? Desenvolva isso no seu texto)
(Por que estão gerando muitas discussões?)
(Por que estão mudando? Deixe isso evidente em seu texto)
(Por que isso acontece? Evidencie isso no seu texto)
(Por que isso ocorre? Discuta isso no texto)
(Por que isso ocorre?)
(Por que isso é importante? Seria interessante discutir isso no texto)
(Por que não seria bom? Apresente a sua reflexão no texto)
(Por que são pontos positivos? Explique)
(Por que é incompatível? Explique isso)
(Por quem? Cite no texto)
(Por quem?)
(Por quê? Apresente essa discussão no texto)
(Por quê? Cite uma causa disso)
(Por quê? Deixe claro no seu texto)
(Por quê? Deixe claro o porquê dessa afirmação)
(Por quê? Deixe evidente no texto)
(Por quê? Deixe isso claro no seu texto)
(Por quê? Deixe isso evidente em seu texto)
(Por quê? Deixe o seu ponto de vista evidente no seu texto)
(Por quê? Desenvolva essa discussão)
(Por quê? Desenvolva essa ideia no seu texto)
(Por quê? Desenvolva essa ideia no texto)
(Por quê? Desenvolva essa ideia)
(Por quê? Desenvolva isso em seu texto)
(Por quê? Desenvolva melhor essa ideia)
(Por quê? Discuta essa ideia no seu texto)
(Por quê? Discuta essa ideia no texto)
(Por quê? Discuta essa ideia)
(Por quê? Discuta isso no seu texto)
(Por quê? Discuta isso no texto)
(Por quê? Evidencie isso no seu texto)
(Por quê? Evidencie no texto)
(Por quê? Evidencie o motivo no texto)
(Por quê? Evidencie o seu ponto de vista no texto)
(Por quê? Exemplifique)
(Por quê? Explique isso no seu texto)
(Por quê? Explique isso no texto)
(Por quê? Explique no texto)
(Por quê? Explique)
(Por quê? Fale sobre isso no seu texto)
(Por quê? Isso não está claro no seu texto)
(Por quê? Justifique)
(Por quê?)
(Portanto ou Por entanto)
(Portanto/ Por entanto)
(Porém)
(Problema de coerência)
(Problema de coesão e coerência)
(Problema na construção de sentido desse período)
(Problema na construção de sentido)
(Problema na construção de sentido. Essa afirmação está confusa. Reflita)
(Problema na construção de sentido. Reelabore essa afirmação)
(Problema na construção de sentido. Reformule esse período)
(Problema na construção de sentido. Reformule)
(Problema na construção de sentido. Reveja coesão e coerência)
(Problema na construção de sentido. Reveja e reformule)
(Problema na contrução de sentido)
(Problemas de coesão e coerência)
(Problemas na construção de sentido)
(Problemas na construção de sentido. Reformule)
(Procure deixar isso evidente em seu texto)
(Procure usar sinônimos)
(Quais agressões?)
(Quais ajustes? Isso não ficou claro no texto)
(Quais atitudes? Deixe-as evidente no texto)
(Quais avaliações nacionais? Cite-as)
(Quais avaliações nacionais?)
(Quais caminhos?)
(Quais consequências?)
(Quais conteúdos importantes? Cite-os)
(Quais crianças?)
(Quais diferenças? Cite-as)
(Quais disciplinas não serão obrigatórias? Fale sobre elas no seu texto e evidencie um possível problema)
(Quais especialistas? Exemplifique)
(Quais expectativas? Cite-as)
(Quais falhas? Cite exemplos)
(Quais grupos?)
(Quais humilhações? Discuta no texto)
(Quais investimentos? Dê exemplos)
(Quais jovens? Os violentos ou os agressores?)
(Quais matérias? Cite-as)
(Quais mecanismos?)
(Quais medidas? Fale sobre isso no texto)
(Quais medidas?)
(Quais melhorias? Cite-as)
(Quais mudanças? Exemplifique)
(Quais mudanças?)
(Quais obstáculos?)
(Quais padrões? Cite-os no texto)
(Quais padrões? Fale sobre eles no texto)
(Quais países? Cite-os)
(Quais pessoas?)
(Quais pontos fundamentais?)
(Quais preocupações?)
(Quais princípios?)
(Quais problemas crônicos?)
(Quais problemas? Cite-os)
(Quais projetos?)
(Quais provas?)
(Quais questões estruturais? Cite-as)
(Quais recursos?)
(Quais seriam as decisões acertadas?)
(Quais seriam essas partes significativas? Exemplifique)
(Quais seriam? Apresente no seu texto)
(Quais seriam? Cite-os)
(Quais seriam? Dê exemplos)
(Quais seriam?)
(Quais são os tipos de ameaças? Deixe evidente no texto)
(Quais são os valores morais certos? Reflita)
(Quais valores positivos? Dê exemplos)
(Quais valores sociais? Cite-os no seu texto)
(Quais valores? Dê exemplos)
(Quais valores?)
(Quais? Assim fica muito vago)
(Quais? Cite alguns exmplos)
(Quais? Cite exemplos)
(Quais? Cite no texto)
(Quais? Cite)
(Quais? Cite-as e explique o porquê)
(Quais? Cite-as)
(Quais? Cite-os no seu texto)
(Quais? Cite-os no texto)
(Quais? Cite-os)
(Quais? Como assim? Esta confuso)
(Quais? Cuidado com esse tipo de colocação. Reformule)
(Quais? Discuta essa ideia no seu texto)
(Quais? Discuta isso em seu texto)
(Quais? Discuta no texto)
(Quais? Dê exemplos)
(Quais? Exemplifique)
(Quais? Não foi citado no texto)
(Quais? Observe que esse tipo de construção feita nesse período é para evocar uma ideia que fora mencionada anteriormente)
(Quais? Perceba que não fora mencionado no texto)
(Quais? Pontue a que você achar de maior relevância)
(Quais? Você não citou no texto)
(Quais?)
(Qual ato? Não está explícito no texto)
(Qual ato? Você não apresenta o que é o bullying)
(Qual crise?)
(Qual dano histórico?)
(Qual dos problemas citados? A evasão escolar ou o desinteresse por determinadas disciplinas?)
(Qual função?)
(Qual humilhação? Dê exemplos)
(Qual ideia)
(Qual impasse? Não está tão evidente no texto)
(Qual impasse?)
(Qual modelo? Fale sobre ele)
(Qual modelo?)
(Qual opção nova?)
(Qual organização? Observe que ficou confuso, vago)
(Qual pesquisa? Apresente no texto)
(Qual proposta?)
(Qual prática?)
(Qual raiz?)
(Qual seria a capacidade física ideal?)
(Qual seria a maneira correta?)
(Qual seria a possível causa disso? É interessante que haja uma discussão sobre isso no seu texto)
(Qual seria o modelo de educação ideal para o Brasil? Fale sobre isso em seu texto)
(Qual seria o padrão social aceito? Discuta isso no texto)
(Qual séria a causa desse resultado? Procure trabalhar isso no seu texto)
(Qual é a forma de conscientizar a sociedade?)
(Qual é o motivo do surgimento desses problemas? Deixe essa informação evidente no seu texto)
(Qual)
(Qual? A primeira ou a segunda?)
(Qual? Deixe essa ideia evidente)
(Qual? Dê exemplo)
(Qual?)
(Quando isso foi dito? Foi dito em que veículo de informação?)
(Quando? Apresente esse fato e explique o porquê disso ter acontecido)
(Que condições são essas? Exemplifique)
(Que consequência futura?)
(Que efeito contrário?)
(Que ideial é esse? Apresente no texto, pois essa informação está vaga)
(Que lei é essa? Quando foi aprovada?)
(Que lei?)
(Que medidas seriam essas?)
(Que mudanças precisam ser feitas?)
(Que padrão? Discuta no texto)
(Que padrões seriam esses?)
(Que pesquisa?)
(Que pessoas seriam?)
(Que seriam? Deixe evidente no texto)
(Que seriam?)
(Que tiipo de melhorias?)
(Que tipo de construção? Isso não está claro no seu texto)
(Que tipo de desigualdade? Deixe isso evidente em seu texto)
(Que tipo de desigualdade? Exemplifique)
(Que tipo de discussões? Desenvolva isso no seu texto)
(Que tipo de evolução, pois, assim, a ideia fica muito ampla)
(Que tipo de ferramenta? Deixe essa ideia clara em seu texto)
(Que tipo de frustação? Cite-a no seu texto)
(Que tipo de impasse? Desenvolva essa ideia)
(Que tipo de incentivo? Exemplifique)
(Que tipo de incentivo? Fale sobre isso no texto)
(Que tipo de instrução? Discuta no texto)
(Que tipo de investimento)
(Que tipo de investimento? Exemplifique)
(Que tipo de investimento?)
(Que tipo de investimento?Cite no texto)
(Que tipo de melhoria? Isso não está explícito no seu texto)
(Que tipo de melhoria? O que está ruim? Evidencie isso no seu texto)
(Que tipo de melhoria?,(Ponto final para o início de um novo perído; problemas de coesão textual)
(Que tipo de mudanças?)
(Que tipo de palestra? Deixe claro no texto)
(Que tipo de projeto? Discuta no texto)
(Que tipo de punição)
(Que tipo de punições?)
(Que tipo de repercussão? Cite-a no seu texto)
(Que tipo de reunião?)
(Que tipo de solução?)
(Que trabalho seria esse?)
(Que valores são esses?)
(Que vídeo? Deixe explícito)
(Que é?)
(Quem são esses indivíduos?)
(Quem são esses muitos?)
(Quem são os maiores interessados?)
(Quem é Hobss?)
(Quem? Cite-os)
(Quem?)
(Redundância/você já usou estas mesmas ideias no paragráfo anterior)
(Reelabore essa afirmação)
(Reflita acerca dessa afirmação)
(Reflita essa afirmação; busque o significado de meritocrácia)
(Reflita sobre essa afirmação)
(Reflita sobre essa afirmação. Esse realmente seria o motivo?)
(Reflita sobre essa colocação)
(Reflita sobre esse tipo de construção, visto que a escola faz "parte do mundo". Reformule essa ideia)
(Reflita sobre isso)
(Reflitta sobre essa afirmação)
(Reformule essa ideia)
(Reformule essa ideia, visto que ela está confusa)
(Reformule esse período)
(Reformule)
(Reformule, pois não ficou claro o que você deseja apontar)
(Reformule. Reveja coesão e coerência)
(Renúncia)
(Repense acerca dessa afirmação)
(Representa a mesma ideia do que é dito anteriormente. Desse modo, essa indormação é dispensável ao texto)
(Reveja essa afirmação)
(Se vírgula)
(Se)
(Sem ":")
(Sem "se")
(Sem aspas)
(Sem dois pontos)
(Sem dois pontos; letra minúscula)
(Sem hífen)
(Sem os dois pontos)
(Sem parênteses)
(Sem ponto de exclamação)
(Sem ponto e vírgula)
(Sem ponto final)
(Sem ponto final, apenas a vírgula)
(Sem ponto final, usa-se vírgula)
(Sem ponto final. Usa-se vírgula)
(Sem ponto final. Use a vírgula)
(Sem travessão)
(Sem vírgla)
(Sem vírgul)
(Sem vírgula)
(Sem vírgula. Usa-se ponto final)
(Sem vírgulas)
(Sem vírgulas. Usa-se dois pontos)
(Será que só agora isso veio à tona? Reflita)
(Será)
(Solucionar de que maneira? Exemplifique)
(Somente o ponto final)
(São exemplos)
(São)
(Só a lei é suficiente?)
(Também deve-se investir em)
(Também)
(Tem-se falado)
(Temas)
(Tente explicar o porquê disso ocorrer)
(Tipo de argumento clichê)
(Tipo de construção clichê)
(Tipo de construção piegas)
(Todavia)
(Trabalhe melhor as ideias deste período)
(Trabalhe melhor essa ideia)
(Trata-se)
(Tratando o quê?)
(Um exemplo seria)
(Um exemplo é o que)
(Um exemplo é)
(Um)
(Usa-se "mesmo" quando se quer atribuir sentido de igualdade)
(Usa-se vírgula ao invés do ponto e vírgula)
(Use mesma quando houver sentindo de igual)
(Use pronomes relativos para evitar o queísmo no texto)
(Uso de primeira pessoa)
(Uso inadequado do ";")
(Uso obrigatório de paragráfo)
(Vale lembrar que o âmbito escolar não se resume em professores e alunos)
(Veja alguns elementos coesivos que podem melhorar a coerência no seu texto)
(Virgula)
(Vitória)
(Você citou apenas uma consequência. Cite as demais)
(Você tem esse tipo de problemas com alunos de sete anos?)
(Víirgula)
(Vírgul)
(Vírgula e letra minúscula)
(Vírgula)
(Vírgula; quais países?)
(Vítimas de quê e de quem? Deixe essa informação evidente no texto)
(Víírgula)
([,] além)
(a – acrescentar)
(a – sem crase)
(a)
(abaixo do esperado)
(abaixo dos)
(abordagem sobre)
(acento no i)
(abrigar)
(acaba)
(aconteceu)
(acredita)
(acredita-se)
(adolescentes)
(aflinge)
(agir por meio de)
(agirmos)
(agravar)
(agressiva e violeta representam a mesma ideia)
(agressão)
(agricultor )
(agricultora)
(ainda bem)
(ainda mais importante)
(ainda mais)
(ainda)
(ajudarão)
(alcunhados de “nem-nem”)
(alega estar)
(alega)
(alguns)
(aliada à)
(almeja)
(anos?)
(antítese)
(análise)
(ao mesmo tempo)
(ao que se refere à quantidades ou frequência de seu uso)
(ao)
(aonde estão)
(aos locais)
(aos quais)
(aos)
(apagões)
(aparentava estar)
(apelativas)
(apenas – acrescentar)
(apenas)
(apesar de que)
(aplicada atualmente)
(apoia)
(aproveitam)
(aproveitar)
(aprovou a lei que)
(as quais)
(as ruas)
(as)
(aspecto)
(assim como)
(assim)
(assim,)
(assinantes)
(assuntos)
(atribui-se)
(atrás)
(atualmente)
(até a)
(até)
(atípica)
(aumentar)
(aumentou)
(autoridades)
(autoritário)
(avanço)
(aí)
(b)
(bem)
(beneficiem)
(benefício)
(bens)
(biologia)
(biomassa)
(bônus)
(c)
(cada)
(calados!)
(caminho)
(capacidade)
(cardeais estão extremamente)
(cargo)
(casamento)
(certa,)
(certos)
(chuva)
(cicatrizar)
(cidade com)
(cidadãos brasileiros)
(cidadãos)
(circunstâncias)
(citados no primeira  )
(cite a fonte desta afirmação)
(coadjuvantes)
(cobrir)
(com a)
(com certeza)
(com crase)
(com essa)
(com esses fatores)
(com indignação)
(com interesses puramente econômicos)
(com o objetivo)
(com o qual)
(com o)
(com ou sem ajuda)
(com que)
(com quem vai se)
(com raras exceções)
(com)
(combustível)
(cometendo)
(cometer)
(cometessem)
(cometidas)
(como a)
(como o)
(como por)
(como sendo)
(como sinal)
(como)
(comum)
(comunicação)
(comunitários)
(comuns)
(comércio)
(concedida a partir)
(conclui-se)
(,)
(-)
(.)
(1)
(2)
(3)
(4)
(:)
(;)
(?)
(A)
(O)
(a)
(b)
(c)
(d)
(e)
(h)
(m)
(n)
(o)
(r)
(s)
(É)
(à)
(á)
(é)
(ê)
(ó)
((;)
(, )
(C )
(Um)
(ao)
(as)
(aí)
(da)
(de)
(do)
(e,)
(em)
(en)
(es)
(ex)
(g1)
(há)
(in)
(ir)
(já)
(m³)
(na)
(no)
(os)
(ou)
(re)
(sa)
(se)
(só)
(um)
(vê)
(às)
(ão)
(***)
(-os)
(-se)
(...)
(???)
(ano)
(aos)
(até)
(bem)
(com)
(cão)
(das)
(diz)
(dos)
(e a)
(e o)
(e é)
(ela)
(era)
(etc)
(foi)
(for)
(ira)
(irá)
(mTa)
(mal)
(mas)
(mau)
(mil)
(más)
(mês)
(ndo)
(nos)
(nte)
(não)
(nós)
(pel)
(por)
(pré)
(que)
(rua)
(sem)
(ser)
(seu)
(sim)
(sua)
(são)
(tal)
(tem)
(ter)
(têm)
(uma)
(vem)
(vez)
(vêm)
(é a)
(, em)
(, já)
(,que)
(Cite)
(Como)
(Eros)
(Esse)
(Esta)
(a de)
(algo)
(alto)
(além)
(anos)
(arma)
(atos)
(bens)
(cada)
(como)
(cria)
(dada)
(dali)
(dava)
(demo)
(deve)
(e as)
(e do)
(elas)
(eles)
(esse)
(esta)
(este)
(está)
(etc.)
(feto)
(fica)
(funk)
(gera)
(há a)
(irão)
(isso)
(lado)
(mais)
(mude)
(mãos)
(nada)
(nove)
(o de)
(onde)
(pais)
(para)
(país)
(pela)
(pelo)
(pode)
(pois)
(poém)
(quem)
(quis)
(quão)
(real)
(saem)
(seja)
(será)
(sete)
(seus)
(sido)
(suas)
(tema)
(tese)
(tido)
(tipo)
(todo)
(traz)
(três)
(vale)
(vive)
(área)
(é um)
(Ainda)
(Ambos)
(Andes)
(Assim)
(Foram)
(Líder)
(Outro)
(Porém)
(Qual?)
(Quem?)
(Temas)
(a fim)
(a ser)
(a ver)
(acaba)
(ainda)
(alega)
(altos)
(andar)
(anos?)
(antes)
(apoia)
(assim)
(atrás)
(até a)
(bônus)
(cargo)
(casos)
(chuva)
(com a)
(com o)
(como:)
(comum)
(crase)
(de se)
(deixa)
(desde)
(dessa)
(devem)
(disso)
(e por)
(e sim)
(ele é)
(enfim)
(entre)
(essas)
(esses)
(estar)
(estes)
(estão)
(fator)
(faz o)
(façam)
(feita)
(feito)
(ficam)
(ficha)
(fiéis)
(fonte)
(foram)
(forem)
(forma)
(fosse)
(fácil)
(gasto)
(geram)
(hajam)
(houve)
(impor)
(levam)
(levar)
(levou)
(líder)
(maior)
(manda)
(marca)
(marco)
(março)
(menos)
(mesmo)
(morte)
(muito)
(média)
(mídia)
(nosso)
(noção)
(nunca)
(o que)
(obtêm)
(outro)
(pague)
(pegos)
(pelos)
(penas)
(perda)
(peças)
(podem)
(poder)
(porém)
(possa)
(quais)
(quase)
(que o)
(que é)
(queda)
(redes)
(ricos)
(rúgbi)
(saúde)
(segue)
(sendo)
(serem)
(seria)
(serve)
(serão)
(sobre)
(somos)
(série)
(tanto)
(temos)
(tenha)
(terem)
(teria)
(terão)
(tomar)
(trará)
(verem)
(vinte)
(vê-se)
(~~;~~)
(Édito)
(áreas)
(única)
( e até)
("funk")
(24.703)
(Acabou)
(Alguns)
(Estado)
(Este é)
(Igreja)
(Lei da)
(Muitos)
(No dia)
(O quê?)
(Outros)
(Passam)
(Piegas)
(Quais?)
(Que é?)
(Também)
(Vírgul)
(a data)
(a mais)
(alguns)
(almeja)
(alunos)
(apenas)
(assim,)
(avanço)
(certa,)
(certos)
(cobrir)
(coisas)
(como a)
(como o)
(compor)
(comuns)
(contra)
(crimes)
(de que)
(de ser)
(deixou)
(devido)
(dúvida)
(e cuja)
(e onde)
(em que)
(em uma)
(espaço)
(estado)
(estava)
(etnias)
(exista)
(existe)
(faz-se)
(filhos)
(foi no)
(fossem)
(física)
(graves)
(grupos)
(hostil)
(hábito)
(ideias)
(imagem)
(início)
(jovens)
(juntos)
(já não)
(kracia)
(kratos)
(lei da)
(magras)
(mas há)
(mesmos)
(motivo)
(muitas)
(muitos)
(multas)
(máximo)
(mínima)
(número)
(o "eu")
(o caso)
(o fato)
(o gado)
(o país)
(o qual)
(o tema)
(ocorre)
(opções)
(origem)
(ou mal)
(ou não)
(ou por)
(outras)
(outros)
(padres)
(papado)
(papéis)
(para o)
(países)
(pensar)
(pessoa)
(porque)
(possui)
(punido)
(quanto)
(que há)
(quinto)
(regras)
(saíram)
(sem um)
(severa)
(sugere)
(talvez)
(também)
(tantos)
(termos)
(tinham)
(típica)
(título)
(uso de)
(vieram)
(voltam)
(várias)
(vítima)
(à esta)
(é que,)
(índice)
(ótimos)
(última)
(último)
( negros)
(, o que)
(A frase)
(Afinal,)
(Algumas)
(Cite-as)
(Cite-os)
(Como no)
(Confuso)
(Sem ":")
(Todavia)
(Virgula)
(Vitória)
(Vírgula)
(a estas)
(a favor)
(a frase)
(a ideia)
(a mídia)
(a terem)
(a água,)
(abrigar)
(aflinge)
(agirmos)
(agravar)
(além da)
(além de)
(análise)
(apagões)
(art. 9º)
(as ruas)
(aspecto)
(atípica)
(bullyng)
(caminho)
(castigo)
(com que)
(cometer)
(confuso)
(correto)
(criança)
(cristão)
(crédito)
(crítico)
(de Papa)
(de suas)
(de voto)
(decisão)
(defende)
(deve se)
(deve-se)
(devemos)
(dificíl)
(direito)
(diálogo)
(domínio)
(durante)
(dívidas)
(e afins)
(e muito)
(e sites)
(e votos)
(em prol)
(escolar)
(espaços)
(estarem)
(exemplo)
(existem)
(externa)
(família)
(fatores)
(fora do)
(governo)
(horária)
(ilícita)
(impacto)
(impunes)
(indigna)
(interno)
(ladrões)
(lavagem)
(levados)
(levando)
(líderes)
(maiores)
(mais se)
(mas sim)
(matéria)
(meio de)
(melhora)
(menos o)
(motivos)
(médicos)
(na qual)
(na área)
(nenhuma)
(ninguém)
(normais)
(notícia)
(notória)
(o "nós")
(o aluno)
(o autor)
(o homem)
(os anos)
(ou mais)
(ou não!)
(pessoas)
(poderia)
(poderão)
(por que)
(porém é)
(preciso)
(prendem)
(prestam)
(privado)
(prática)
(própria)
(punidos)
(pública)
(que foi)
(que são)
(que ter)
(ranking)
(receber)
(reforma)
(retirou)
(revisar)
(sistema)
(somente)
(startup)
(surgirá)
(temendo)
(tráfico)
(visível)
(vírgula)
(vítimas)
(~~com~~)
(étnicos)
(últimas)
(últimos)
( brancos)
( frizou )
((Vírgula)
(A partir)
(Católica)
(Colômbia)
(Conselho)
(Consuela)
(Denatran)
(Diante d)
(Durkheim)
(Enfrente)
(Gregório)
(Houve um)
(MedioTec)
(Muito se)
(O acesso)
(Por quê?)
(Que lei?)
(Renúncia)
(Rousseff)
(Sem "se")
(Sinafese)
(Trata-se)
(Vaticano)
(Víirgula)
(Víírgula)
([,] além)
(a doença)
(a escola)
(a fim de)
(a medida)
(a partir)
(a pessoa)
(a rotina)
(acredita)
(agressão)
(ajudarão)
(aliada à)
(ambiente)
(amenizar)
(antítese)
(ao invés)
(artigo 2)
(as quais)
(assuntos)
(aumentar)
(aumentou)
(biologia)
(biomassa)
(bullying)
(calados!)
(católica)
(cidadãos)
(clerical)
(com essa)
(como por)
(comércio)
(contrair)
(crescido)
(crianças)
(cristãos)
(críticas)
(daqueles)
(de armas)
(de fiéis)
(de idade)
(declarar)
(declínio)
(deixando)
(deixaram)
(deve ser)
(deve ter)
(deve vir)
(deveriam)
(dinheiro)
(direitos)
(disseram)
(ditadura)
(diversos)
(do mesmo)
(do que é)
(do vírus)
(e muito!)
(em geral)
(em reais)
(em tempo)
(em vigor)
(equívoco)
(escravos)
(evitados)
(evitem a)
(evolução)
(examinar)
(extinção)
(falta de)
(fetos de)
(frente a)
(gasolina)
(gravidez)
(heresias)
(horários)
(houveram)
(inegável)
(infantis)
(inserção)
(internet)
(intrigam)
(isto é(,)
(linear ?)
(mundiais)
(mutantes)
(na China)
(na Índia)
(negativo)
(netfobia)
(o "nerd")
(o Brasil)
(o Estado)
(o avanço)
(o índice)
(ocorrido)
(ofereceu)
(olharmos)
(opiniões)
(os quais)
(ou falta)
(ou mamãe)
(para que)
(passaram)
(país que)
(pessoais)
(petróleo)
(polêmica)
(polêmico)
(política)
(político)
(por Deus)
(por eles)
(por meio)
(possuíam)
(possível)
(praticam)
(praticar)
(presídio)
(privados)
(problema)
(promessa)
(próprias)
(próximos)
(públicas)
(quarenta)
(que haja)
(quiserem)
(realizar)
(renuncia)
(renúncia)
(repórter)
(rodovias)
(ruas das)
(salários)
(se alaga)
(sem "se")
(silêncio)
(singular)
(situação)
(sozinhos)
(tal fato)
(todos os)
(torna-se)
(tratando)
(trazê-la)
(uma arma)
(unidades)
(valentão)
(visíveis)
(vivência)
( alienada)
( brigas(,)
( que está)
(Denúncias)
(Diante de)
(Essas são)
(Executivo)
(Há também)
(Igualdade)
(No Brasil)
(O Uruguai)
(Paragráfo)
(Por quem?)
(Reformule)
(Sem aspas)
(Sem hífen)
(a atender)
(a reforma)
(aceitação)
(aconteceu)
(ainda bem)
(ameaçados)
(americano)
(aos quais)
(as Etecs )
(benefício)
(boa parte)
(casamento)
(católicos)
(com crase)
(cometendo)
(cometidas)
(conseguiu)
(constatar)
(construir)
(contrário)
(das vezes)
(de chuvas)
(de drogas)
(de que na)
(denúncias)
(depois de)
(desde que)
(desespero)
(despertar)
(despertou)
(devem ser)
(dia a dia)
(diante do)
(didáticos)
(direito a)
(discussão)
(disque100)
(distritos)
(dividindo)
(durante o)
(e mataram)
(e pessoas)
(em frente)
(encontram)
(esposo (a)
(eventuais)
(evidência)
(evoluírem)
(executivo)
(fair play)
(fazendo-o)
(fervoroso)
(fevereiro)
(fictícios)
(foi feito)
(frustrada)
(grupos de)
(hegemonia)
(incentivo)
(inclusive)
(incomodar)
(indivíduo)
(informada)
(infrações)
(infringir)
(interagem)
(início do)
(levando à)
(liberados)
(maiúscula)
(maíusculo)
(medievais)
(mesmo que)
(minúscula)
(minúsculo)
(monopólio)
(mostraram)
(município)
(no Brasil)
(no México)
(no caso(,)
(no mínimo)
(novamente)
(o consumo)
(o direito)
(o governo)
(o próprio)
(o virtual)
(os alunos)
(ou sempre)
(para eles)
(parte dos)
(passe por)
(pedofilia)
(pedófilos)
(polêmicas)
(políticas)
(políticos)
(por cento)
(por saber)
(possíveis)
(presídios)
(primeiros)
(problemas)
(publicado)
(quanto de)
(que têm o)
(referente)
(relação a)
(relação à)
(religiosa)
(renunciar)
(renúncia,)
(se houver)
(sem crase)
(sem o "e")
(semanário)
(sendo que)
(seria que)
(seu filho)
(situações)
(também na)
(tornassem)
(tornem-se)
(tragédia?)
(trouxesse)
(três anos)
(utilidade)
(violentos)
(água doce)
(é preciso)
(é que são)
( televisão)
(A campanha)
(Desenvolva)
(Desse modo)
(Deste modo)
(Deve ainda)
(Dê exemplo)
(Iluminismo)
(Muitas das)
(Networking)
(O filósofo)
(Observando)
(Ponto fina)
(Ponto fnal)
(Presidente)
(Qual ideia)
(Qual raiz?)
(Se vírgula)
(Sem vírgla)
(Sem vírgul)
(Socialismo)
(a internet)
(a renúncia)
(abaixo dos)
(ainda mais)
(aos locais)
(apelativas)
(aproveitam)
(aproveitar)
(artifícios)
(as dívidas)
(as pessoas)
(assim como)
(assinantes)
(atribui-se)
(atualmente)
(beneficiem)
(capacidade)
(cicatrizar)
(cidade com)
(com o qual)
(cometessem)
(como sendo)
(como sinal)
(computador)
(conclui-se)
(confirmada)
(consciente)
(contribuem)
(convencera)
(de um modo)
(debilidade)
(decapitado)
(define que)
(democracia)
(desvínculo)
(deve haver)
(discussões)
(discuti-lo)
(disposição)
(distinguir)
(divulgados)
(do fato de)
(e ainda há)
(e com quem)
(e possível)
(e sinônimo)
(elaboradas)
(em jornais)
(em relação)
(em vista o)
(empresário)
(específica)
(etc [etc.])
(eticamente)
(felicidade)
(fetichismo)
(floresciam)
(havia dito)
(imparciais)
(indivíduos)
(influência)
(informação)
(irrestrito)
(judiciário)
(lei 10.826)
(maioridade)
(mais tarde)
(melhora no)
(mudança na)
(na prática)
(nas mortes)
(necessário)
(no fato de)
(não apenas)
(o ministro)
(o problema)
(os pobres!)
(os poderes)
(ou favelas)
(ou frontal)
(ou marido )
(palestinos)
(para todos)
(política é)
(praticados)
(praticou o)
(preocupado)
(presidente)
(projeto de)
(proposital)
(provocaram)
(pós-guerra)
(querem que)
(realizadas)
(redudância)
(relevantes)
(renuncia a)
(retornando)
(revela que)
(revertidos)
(ruas pelas)
(se mudaram)
(seguidores)
(sem acento)
(sem limite)
(seu marido)
(sobre qual)
(superior a)
(século XIX)
(também têm)
(temos como)
(tenha sido)
(traficante)
(uma edição)
(universais)
(vestibular)
(vinte anos)
(é algo que)
(é possível)
( reificação)
(, de acordo)
(,como dizia)
(Cinco Olhos)
(Cite alguns)
(Cite o nome)
(Como assim?)
(De que modo)
(Dois pontos)
(Dê exemplos)
(Instituição)
(Isso o quê?)
(O vírus foi)
(Ponto final)
(Pão e Circo)
(Quais? Cite)
(Qual crise?)
(Que seriam?)
(Redundância)
(Sem vírgula)
(Sendo assim)
(Senso comum)
(a Grande SP)
(a aquisição)
(a corrupção)
(a juventude)
(a que foram)
(a separação)
(a sociedade)
(a tv aberta)
(acento no i)
(acredita-se)
(agricultor )
(agricultora)
(alega estar)
(aniversário)
(aonde estão)
(as crianças)
(as mulheres)
(até 6 meses)
(autoridades)
(autoritário)
(brasileiros)
(brincadeira)
(carro, moto)
(casos raros)
(com certeza)
(combustível)
(como música)
(comunicação)
(consciência)
(contradição)
(contribuído)
(conveniente)
(cumprimento)
(da internet)
(da renúncia)
(de esportes)
(de modo que)
(de trabalho)
(declarações)
(decreta que)
(desenvolver)
(determina a)
(do machismo)
(dois pontos)
(e desejável)
(elaborá-las)
(eles mesmos)
(em destaque)
(em educação)
(encontra-se)
(entre aspas)
(fica oculto)
(importância)
(individuais)
(infringirem)
(intolerável)
(legislativo)
(momentâneos)
(nas cidades)
(nordestinos)
(não aceita?)
(não agredir)
(nível médio)
(o "caipira")
(o Brasil em)
(o contrário)
(o número de)
(o que afeta)
(o que causa)
(o religioso)
(obrigatório)
(ou até mais)
(ou até pior)
(ou invadido)
(ou nem isso)
(pelas quais)
(pergunta-se)
(períodos de)
(pesquisa do)
(podem tomar)
(poderia ser)
(ponto final)
(por exemplo)
(por meio de)
(por meio do)
(preocuparem)
(psicológica)
(psicológico)
(pão e circo)
(quanto para)
(que ocorreu)
(que se faça)
(redundância)
(remuneração)
(renunciaram)
(responsável)
(ressaltaram)
(se passaram)
(se tornasse)
(sem assunto)
(sem atração)
(sem vírgula)
(sendo assim)
(seus alunos)
(sistema que)
(socialmente)
(sua eleição)
(sua maioria)
(suficientes)
(surpreendeu)
(século XVII)
(teve início)
(tradicional)
(traficantes)
(têm buscado)
(um palhaço!)
(um total de)
(uma parcela)
(~~ruas (com)
(à pedofilia)
( zidovudina )
((Sem vírgula)
(.ponto final)
(A internet é)
(A notícia de)
(Cantareira, )
(Constituição)
(De que modo?)
(Em relação à)
(Especulações)
(Exemplifique)
(Falou o quê?)
(Fraternidade)
(Há no Brasil)
(Ideia clichê)
(Muitos quem?)
(Outros quem?)
(Qual função?)
(Qual modelo?)
(Recentemente)
(Sem vírgulas)
(São exemplos)
(Um exemplo é)
(a espionagem)
(adolescentes)
(as enchentes)
(bem coletivo)
(coadjuvantes)
(como o carro)
(comunitários)
(conquistados)
(consequência)
(consideradas)
(contribuindo)
(da população)
(desconfiança)
(desidratados)
(desigualdade)
(distribuição)
(e condenados)
(e financiado)
(e há rumores)
(em Nova York)
(em conta que)
(em discussão)
(em episódios)
(em relação a)
(em relação à)
(empregadores)
(erroneamente)
(escolaridade)
(experiências)
(final do ano)
(guerra santa)
(homossexuais)
(infelizmente)
(microcefalia)
(no século XX)
(não passiva!)
(não-fumantes)
(o "orelhudo")
(o crussifixo)
(o vírus zika)
(os filósofos)
(os políticos)
(os shoppings)
(ou de querer)
(para 16 anos)
(pois família)
(problemas no)
(prostituição)
(providências)
(quase sempre)
(que deve ser)
(que são boas)
(quer dizer(,)
(recentemente)
(rede privada)
(rede pública)
(relaciona-se)
(relacionadas)
(responsáveis)
(sala de aula)
(se não todos)
(sendo assim,)
(ser colocado)
(tanto,quanto)
(teoricamente)
(the slutwalk)
(um empecilho)
(~~ no PISA~~)
(á anos atrás)
(é necessário)
(é preciso se)
( acento agudo)
( os especiais)
(, por exemplo)
(, uma vez que)
(. ponto final)
(Cite exemplos)
(Dê um exemplo)
(Esta renúncia)
(O – maiúsculo)
(Os apoiadores)
(Os rolezinhos)
(Pena de Morte)
(Quais grupos?)
(Quais provas?)
(Quais seriam?)
(Qual impasse?)
(Qual prática?)
(Que pesquisa?)
(Quem é Hobss?)
(Quem? Cite-os)
(Sem travessão)
(Tem-se falado)
(a ação humana)
(a diversidade)
(a obtenção de)
(a – sem crase)
(apesar de que)
(avançada, não)
(como o Brasil)
(consequências)
(de acordo com)
(despreparados)
(deve haver um)
(e até trágico)
(e o Executivo)
(e representam)
(e um problema)
(efeito dominó)
(em Bangladesh)
(energia solar)
(ensimesmar-se)
(etnocentrismo)
(evidenciam-se)
(exemplares do)
(ficar somente)
(grampeamentos)
(hidrelétricas)
(incentivá-las)
(investimentos)
(já garantidos)
(manifestações)
(na atualidade)
(na disciplina)
(norte e leste)
(não tem crase)
(originalmente)
(os membros da)
(ou a desgraça)
(ou até nenhum)
(para que haja)
(países como a)
(peixes e aves)
(pelo menos um)
(por parte dos)
(por problemas)
(praça pública)
(primeiro Papa)
(provavelmente)
(quando ocorre)
(que os espera)
(que se sucede)
(redes sociais)
(se é que tem!)
(senhor feudal)
(ser aplicadas)
(ser realizada)
(será a reação)
(texto anônimo)
(use a vírgula)
(uso de vígula)
(vento, sol...)
(voltou à tona)
(vários países)
(~~Cid Gomes~~)
(~~e~~ maioria)
(~~ibeb~~ ideb)
(África do Sul)
( pele vermelha)
(, entre outras)
(Cite uma fonte)
(Entre vírgulas)
(Letra maiúsula)
(Não há "deste")
(O que é coisa?)
(Quais medidas?)
(Quais pessoas?)
(Quais valores?)
(Quais? Cite-as)
(Quais? Cite-os)
(Qual proposta?)
(Sem parênteses)
(a aprovação de)
(antigo 2º grau)
(ao mesmo tempo)
(após a escrita)
(artes marciais)
(atividades fim)
(autoridade mor)
(branca e negra)
(circunstâncias)
(com indignação)
(com o objetivo)
(como informado)
(constantemente)
(daí o seu nome)
(de face á face)
(de privacidade)
(decorrentes de)
(desde que haja)
(dificuldade na)
(direitos civis)
(divulgado pela)
(dizer o mínimo)
(e a criação de)
(e afirmava que)
(energia eólica)
(essa juventude)
(essa liberdade)
(essas atitudes)
(foram chamados)
(física e moral)
(guerras santas)
(irmão ou filho)
(mais populares)
(mais violentas)
(manifestava-se)
(muito trabalho)
(não há virgula)
(não há vírgula)
(o que é errado)
(oito por cento)
(os brasileiros)
(os terroristas)
(ou aculturação)
(ou inexistente)
(ou quase todos)
(pelos cidadãos)
(penitenciárias)
(principalmente)
(que deve haver)
(que não mentem)
(que não é dele)
(que, em vez de)
(quinhentos mil)
(renuncia a seu)
(representá-los)
(salvo exceções)
(salário mínimo)
(sempre lotados)
(teologicamente)
(ter acobertado)
(uso da vírgula)
(uso de vírgula)
(veio a falecer)
( principalmente)
(, o que implica)
(Agora, frente à)
(De que maneira?)
(Deus odeia gays)
(Deveria ou deve)
(Letra maiúscula)
(Letra minúscula)
(Mas o “por que”)
(Nem todos quem?)
(O primeiro caso)
(Os outros quem?)
(Período confuso)
(Quais caminhos?)
(Quais crianças?)
(Quais mudanças?)
(Quais projetos?)
(Quais recursos?)
(Sem dois pontos)
(Sem ponto final)
(Tratando o quê?)
(a publicação de)
(a – acrescentar)
(abordagem sobre)
(amigos virtuais)
(aptidão natural)
(assim como a de)
(ação ou omissão)
(chefe de estado)
(com quem vai se)
(como consideram)
(comprometidos a)
(conservadorismo)
(corrupção ativa)
(de acordo com a)
(desta gravidade)
(devotos, ou não)
(discutidos pela)
(e especialmente)
(e levá-las para)
(e o catolicismo)
(especialmente o)
(funções típicas)
(inclusão social)
(letra maiúscula)
(letra minúscula)
(longe de acabar)
(maioria armados)
(mas para outros)
(no fornecimento)
(os quais rompem)
(ou homoafetivos)
(para os adultos)
(período confuso)
(pobres e negros)
(pronunciamentos)
(próximo de rios)
(que fizeram com)
(serão reduzidas)
(seus candidatos)
(socioeducativas)
(socioeducativos)
(subliminarmente)
(são necessários)
(símbolo cristão)
(tal qual a Aids)
(use a 3ª pessoa)
(visão e audição)
(à – use a crase)
(A experiência do)
(Ditadura Militar)
(Expressão clichê)
(Não é necessário)
(Perseguir o quê?)
(Quais agressões?)
(Qual opção nova?)
(Qual? Dê exemplo)
(Um exemplo seria)
(a evasão escolar)
(agir por meio de)
(além do nordeste)
(ao vínculo entre)
(aparentava estar)
(civil e criminal)
(com ou sem ajuda)
(como no valor da)
(de 5 a 10 gramas)
(de maioria negra)
(dois mil e treze)
(drogas, assaltos)
(e acredita nelas)
(em falta de água)
(em julho de 2014)
(este dispositivo)
(está relacionada)
(ex. poliomielite)
(existem exceções)
(fim da corrupção)
(fraca e instável)
(grupo terrorista)
(hipercapitalismo)
(homossexualidade)
(identidade falsa)
(lê-se coligações)
(mix martial arts)
(não me arrependo)
(o relacionamento)
(ocorrem inúmeros)
(origem e destino)
(ou acomodaram-se)
(ou libera poucos)
(ou simpatizantes)
(outro pontífice,)
(perceberemos que)
(práticas sociais)
(que está lá fora)
(rainhas do Egito)
(realidade social)
(sociedade linear)
(sustentabilidade)
(são incentivadas)
(várias polêmicas)
(à – uso da crase)
(à – uso de crase)
(é importante que)
(é imprescindível)
(Cotas para negros)
(Explique o porquê)
(Isso é reflexo da)
(O que são coisas?)
(Por quê? Explique)
(Quais mecanismos?)
(Quais obstáculos?)
(Quais princípios?)
(a admissibilidade)
(as atividades-fim)
(as pessoas possam)
(até mesmo decente)
(colegas de classe)
(com esses fatores)
(como consequência)
(congestionamentos)
(contra o processo)
(corrupção passiva)
(da manhã e almoço)
(de forma moderada)
(de freiar o ciclo)
(debilidade física)
(diesel e gasolina)
(dinheiro este que)
(e única até então)
(energica elétrica)
(facebook, twiter )
(homens e mulheres)
(intenção de matar)
(nas redes sociais)
(no aspecto de que)
(o chamado plano B)
(o governo federal)
(o que possilibita)
(orientação sexual)
(ou ser alvo dela!)
(ou talvez mocinho)
(palavra excedente)
(país desenvolvido)
(pois colabora com)
(por meio da mídia)
(portanto sub-raça)
(que querem lucrar)
(realizado em 2010)
(religião católica)
(renuncia ao cargo)
(rádio, cinema, TV)
(segundo pesquisas)
(será possível que)
(serão vencedores.)
(são cada vez mais)
(terceira dimensão)
(termo entre aspas)
(trecho incoerente)
(uso desnecessário)
(à Igreja Católica)
(às - uso de crase)
(últimas denúncias)
( e o não declarado)
(1970/1978a, p. 287)
(A guerra invisível)
(Alguns deles quem?)
(Como assim? Quais?)
(Marca de oralidade)
(O fim dos problema)
(Quais? Dê exemplos)
(Reflita sobre isso)
(Sem os dois pontos)
(Um exemplo é o que)
(a menos que queira)
(abaixo do esperado)
(anorexia e bulimia)
(boatos da internet)
(bully = “Valentão”)
(concedida a partir)
(de 25 para 21 anos)
(de qualquer gênero)
(decpitando pessoas)
(defendendo a ideia)
(demandam da pessoa)
(dessas substâncias)
(devem ser firmadas)
(direitos e deveres)
(dióxido de carbono)
(do Governo Federal)
(e dos adolescentes)
(energia geotérmica)
(energia maremotriz)
(escritor solitário)
(espionagem virtual)
(esquerda e direita)
(esses sim culpados)
(folha de são paulo)
(inclusive ilícitas)
(menores de 18 anos)
(movimentos sociais)
(natural de Alagoas)
(negros e indígenas)
(não tirar vermelho)
(o messias islâmico)
(ou sendo inseridas)
(pecado contra Deus)
(pode fazer com que)
(poluição das águas)
(por ambos os sexos)
(presidente dos EUA)
(proibir ou apoiar?)
(que foi manipulada)
(que nunca teve fim)
(que poderiam ser(,)
(renúncia repentina)
(risco de ser morto)
(sancionado em 1940)
(serviços indevidos)
(sociedade e Estado)
(uso de ponto final)
(águas subterrâneas)
(é importante saber)
(é o mais duradouro)
( contra ou a favor )
(Apresente esse fato)
(Coloque entre aspas)
(Dê um exemplo disso)
(Isso pode ser feito)
(Marcas de oralidade)
(Por quê? Justifique)
(Quais preocupações?)
(Quais? Exemplifique)
(Que pessoas seriam?)
(Que tipo de punição)
(Sem ponto e vírgula)
(Todos pela Educação)
(aplicada atualmente)
(avaliados pelo PISA)
(como nas monarquias)
(como os agrotóxicos)
(considerados sábios)
(decreto dos plebeus)
(desigualdade racial)
(disso, é importante)
(esquerda ou direita)
(fotos,vídeos,música)
(fundador da Vitacon)
(intoxicação e morte)
(início da República)
(medidas provisórias)
(ministro da fazenda)
(na classe mais alta)
(na escolha do povo )
(no período colonial)
(não inclusão na CLT)
(não podem se fundir)
(número do candidato)
(o golpe republicano)
(organização interna)
(os membros do clero)
(para a nova escolha)
(para que não o faça)
(políticos e sociais)
(por terem cometidos)
(predadores naturais)
(progresso econômico)
(públicas e privadas)
(quando bem cuidados)
(que se trata de uma)
(súmulas vinculantes)
(tributária, agrária)
(utilize a 3ª pessoa)
(utilize outro termo)
(visando ser maioria)
(à – utilize a crase)
(Afirmação equivocada)
(Cite alguns exemplos)
(O governo deve criar)
(Por que isso ocorre?)
(Quais consequências?)
(Quais? Cite exemplos)
(Quais? Cite no texto)
(Qual dano histórico?)
(Que tipo de reunião?)
(Que tipo de solução?)
(Reformule essa ideia)
(alguns trabalhadores)
(alunos e professores)
(apenas – acrescentar)
(até então temos Papa)
(bastante reprováveis)
(caminho do guerreiro)
(como antes acontecia)
(como compra de votos)
(como um resfriamento)
(de forma destacada(,)
(de qualquer religião)
(debilidade e cansaço)
(devem ser realizados)
(disso, policiais são)
(doações e propaganda)
(e / retire a vírgula)
(espaço e professores)
(estarem em evidência)
(estima-se 20 milhões)
(ideia confusa.Revise)
(inclusive a imprensa)
(instituição católica)
(lésbicas e travestis)
(na maioria dos casos)
(não apenas políticos)
(o núcleo na história)
(o sistema Cantareira)
(os menos favorecidos)
(para ambas as partes)
(penas mais severas a)
(pode torná-la finita)
(possível por meio de)
(presidenta do Brasil)
(públicos ou privados)
(que já não é pequena)
(real problema social)
(símbolos de respeito)
(um número suportável)
(vinte três por cento)
(é um aprendizado que)
( "amigos de crianças")
(Afeta de que maneira?)
(Com – letra maiúscula)
(Como assim? Reformule)
(Desenvolva essa ideia)
(Fale mais sobre isso;)
(Fale sobre a proposta)
(Muitos quem? Por quê?)
(Período muito extenso)
(Por quê? Exemplifique)
(Portanto/ Por entanto)
(Problema de coerência)
(Quais países? Cite-os)
(Quais seriam? Cite-os)
(Que efeito contrário?)
(Que tipo de mudanças?)
(Que tipo de punições?)
(Reveja essa afirmação)
(Uso inadequado do ";")
(ainda mais importante)
(casamento homossexual)
(citados no primeira  )
(como muitos especulam)
(de iniciativa popular)
(dependendo da criação)
(dinheiro, patrimônio )
(e seus representantes)
(em quaisquer que seja)
(embora pouco didática)
(empregados/sindicatos)
(especialmente a elite)
(grandes pecuriaristas)
(há exceções, é claro!)
(instituições públicas)
(mesmo sem autorização)
(ministro da Educação,)
(o uso do preservativo)
(ou finge não perceber)
(ou projetos de homens)
(ou pseudointelectuais)
(políticas e jurídicas)
(principalmente jovens)
(proveniente do metano)
(que acabou de comprar)
(que também é um dever)
(sabemos que é difícil)
(shopping~~s~~ centers)
(trabalhista, familiar)
(uma pré-copa do mundo)
(à base de duras lutas)
(Afetar em que sentido?)
(Cuidado ao generalizar)
(Explique a razão disso)
(Falhar em que sentido?)
(Informação dispensável)
(Iniciação de paragráfo)
(O que você quis dizer?)
(Observe essa afirmação)
(Procure usar sinônimos)
(Que valores são esses?)
(Quem são esses muitos?)
(Reformule esse período)
(Só a lei é suficiente?)
(Uso de primeira pessoa)
(Vírgula; quais países?)
(a favor deles próprios)
(a lei da terceirização)
(de forma significativa)
(dias extracurriculares)
(informação dispensável)
(levada em consideração)
(melhore esse argumento)
(o assunto mais tratado)
(o número de automóveis)
(o que sempre aconteceu)
(onde a maioria é negra)
(onde só o rico tem vez)
(ou possui descendência)
(ou se lembra vagamente)
(ou seja, os mais ricos)
(ou toxinas bacterianas)
(para a formação de uma)
(perdeu parte da orelha)
(praticante de bullying)
(praças públicas gregas)
(relacionadas à miséria)
(representando a gestão)
(representantes do povo)
(roubo seguido de morte)
(se é que eles existem!)
(será necessário tomar )
(sistema único de saúde)
(social e governamental)
(só que em larga escala)
(tal como já está sendo)
(uso de letra maiúscula)
(uso de letra minúscula)
(à nação e a corrompend)
( em casos de corrupção )
( força aerea brasileira)
(Cuidado ao afirmar isso)
(Dê exemplos de mudanças)
(Evite o piegas no texto)
(Explique o porquê disso)
(Parágrafo sem coerência)
(Por quem? Cite no texto)
(Portanto ou Por entanto)
(Quais matérias? Cite-as)
(Quais? Cite-os no texto)
(Quais? Discuta no texto)
(Que tiipo de melhorias?)
(Sem ponto de exclamação)
(aquecimento da economia)
(através de uma pesquisa)
(entre elas o consumismo)
(essa quantidade de água)
(longe de áreas de risco)
(medidor da desigualdade)
(mesmo inconscientemente)
(na verdade, por séculos)
(nacionais de pelo menos)
(no caso quem pagar mais)
(não são mais uma opção,)
(termo esdrujulo/ em vão)
("bullying" pela internet)
(Afetaria de que maneira?)
(Como? Apresente no texto)
(Departamento de Imprensa)
(Deveria ou deve? Reflita)
(Movimento dos sem terras)
(Quais melhorias? Cite-as)
(Quais problemas? Cite-os)
(Que consequência futura?)
(Que tipo de investimento)
(Que trabalho seria esse?)
(Reelabore essa afirmação)
(Tipo de argumento clichê)
(parágrafo sem fundamento)
(Apresenta ideais confusas)
(Apresente pontos em comum)
(Como assim? Ficou confuso)
(Desenvolva essa afirmação)
(Exemplifique; ponto final)
(Fale sobre essas mudanças)
(Intimidar de que maneira?)
(Não – com letra maiúscula)
(Quais diferenças? Cite-as)
(Quais problemas crônicos?)
(Quais seriam? Dê exemplos)
(Que medidas seriam essas?)
(Que padrões seriam esses?)
(Que tipo de investimento?)
(Tipo de construção clichê)
(Tipo de construção piegas)
(Vírgula e letra minúscula)
(como muitos alegam (Quem?)
(primeira pessoa do plural)
( "por isso" entre vírgulas)
(Desenvolva mais essa ideia)
(Explique melhor essa ideia)
(Incentivar de que maneira?)
(Por quê? Explique no texto)
(Quais pontos fundamentais?)
(Quais valores? Dê exemplos)
(Quais? Cite alguns exmplos)
(Que vídeo? Deixe explícito)
(Quem são esses indivíduos?)
(Trabalhe melhor essa ideia)
(não deve estar entre aspas)
(à – nesse caso use a crase)
(Como assim? Não ficou claro)
(De que forma isso acontece?)
(Discuta essa ideia no texto)
(Fale mais sobre esse acordo)
(Não está explícito no texto)
(Por quê? Discuta essa ideia)
(Por quê? Evidencie no texto)
(Quais avaliações nacionais?)
(Quais expectativas? Cite-as)
(Quais falhas? Cite exemplos)
(Quais? Cite-os no seu texto)
(Qual modelo? Fale sobre ele)
(Como assim? Discuta no texto)
(,)
(Desenvolva melhor essa ideia)
(Dê exemplos dessas agressões)
(Dê exemplos dessas situações)
(Fale sobre isso no seu texto)
(Parágrafo confuso. Reformule)
(Pesquisa realizada por quem?)
(Quais mudanças? Exemplifique)
(Quais? Assim fica muito vago)
(Qual humilhação? Dê exemplos)
(Que padrão? Discuta no texto)
(Reflita sobre essa afirmação)
(Reflita sobre essa colocação)
(Uso obrigatório de paragráfo)
(cite a fonte desta afirmação)
(nesse caso não use a vírgula)
(não entendi o que quis dizer)
(Conscientizar de que maneira?)
(Expressão clichê. Sem vírgula)
(Por quê? Cite uma causa disso)
(Qual seria a maneira correta?)
(Reflitta sobre essa afirmação)
(entre aspas – “liberalização”)
(não se usa vírgula neste caso)
(Cite esses países no seu texto)
(Como assim? Evidencie no texto)
(Cuidado ao fazer generaliações)
(Cuidado com algumas afirmações)
(Desenvolva essa ideia no texto)
(Entre aspas pois é uma citação)
(Não está evidente no seu texto)
(Onde isso ocorreu? Dê exemplos)
(Por quê? Desenvolva essa ideia)
(Por quê? Discuta isso no texto)
(Problema de coesão e coerência)
(Quais? Não foi citado no texto)
(Quais? Você não citou no texto)
(Qual? A primeira ou a segunda?)
(Reflita acerca dessa afirmação)
(Repense acerca dessa afirmação)
(Sem ponto final. Use a vírgula)
(sem vírgula, use o ponto final)
(Cuidado ao fazer essa afirmação)
(Cuidado ao fazer generalizações)
(Explique meslhor essa afirmação)
(Explique o porquê disso ocorrer)
(Explique que reforma seria essa)
(Por quê? Explique isso no texto)
(Problemas de coesão e coerência)
(Quais padrões? Cite-os no texto)
(Quais? Como assim? Esta confuso)
(Qual? Deixe essa ideia evidente)
(Sem ponto final, usa-se vírgula)
(Sem ponto final. Usa-se vírgula)
(Sem vírgula. Usa-se ponto final)
(Cite exemplos do que seria ideal)
(Como assim? A ideia está confusa)
(Como assim? Reformule essa ideia)
(Cuidado com esse termo "invadir")
(Dia – inicie com letra maiúscula)
(Discutida por quem? Exemplifique)
(Evidencie isso por meio de fatos)
(Exemplifique e discuta as causas)
(Isso não está claro no seu texto)
(Isso não está explícito no texto)
(Isso não está tão claro no texto)
(Por que ele é defasado? Explique)
(Por quê? Deixe evidente no texto)
(Problema na contrução de sentido)
(Quais investimentos? Dê exemplos)
(Quais? Discuta isso em seu texto)
(Sem dois pontos; letra minúscula)
(Sem vírgulas. Usa-se dois pontos)
(período confuso, articule melhor)
(Como assim? Desenvolva essa ideia)
(Como assim? Estamos no século XXI)
(Como asssim? Reformule essa ideia)
(Dê exemplos desses atos violentos)
(Explique o porquê dessa afirmação)
(Fale mais sobre isso no seu texto)
(Fale um pouco mais desses padrões)
(Por quê? Deixe claro no seu texto)
(Problema na construção de sentido)
(Quais especialistas? Exemplifique)
(Qual pesquisa? Apresente no texto)
(Que mudanças precisam ser feitas?)
(Quem são os maiores interessados?)
(Sem ponto final, apenas a vírgula)
(não se utiliza vírgula neste caso)
(não é necessário o uso de vírgula)
(Cuidado ao fazer certas afirmações)
(Cuidado com esse tipo de afirmação)
(De que maneira isso seria possível)
(Dê exemplos desse tipo de agressão)
(Explique o porquê de tal afirmação)
(Falta de conhecimento sobre o quê?)
(O quê? Evidencie isso no seu texto)
(Período com ideias confusas.Revise)
(Por quê? Desenvolva essa discussão)
(Por quê? Discuta isso no seu texto)
(Problemas na construção de sentido)
(Quais? Cite-as e explique o porquê)
(Cuidado ao fazer algumas afirmações)
(Descreva essa situação em seu texto)
(Explique o porquê de ter dado certo)
(Fale sobre o porquê dessa discussão)
(Isso não está tão evidente no texto)
(Isso não ocorrerá em todos os casos)
(O que seria uma conversa relevante?)
(O que seria uma sociedade evoluída?)
(Por quê? Explique isso no seu texto)
(Quais avaliações nacionais? Cite-as)
(Quais humilhações? Discuta no texto)
(Quais questões estruturais? Cite-as)
(Quais seriam as decisões acertadas?)
(Que seriam? Deixe evidente no texto)
(Que tipo de incentivo? Exemplifique)
(Ratzinger – acrescentar o sobrenome)
(quais as fontes que comprovam isso?)
(Como assim? Reformule essa afirmação)
(Deixe o motivo evidente no seu texto)
(Desenvolva as ideias desse parágrafo)
(Desenvolva essa ideia e exemplifique)
(Ele, – com letra maiúscula e vírgula)
(Não ficou claro. Refaça esse período)
(O que seria essa verdadeira mudança?)
(Por quê? Discuta essa ideia no texto)
(Por quê? Evidencie isso no seu texto)
(Por quê? Evidencie o motivo no texto)
(Quais conteúdos importantes? Cite-os)
(Quais seriam? Apresente no seu texto)
(Quais valores positivos? Dê exemplos)
(Que lei é essa? Quando foi aprovada?)
(Reformule. Reveja coesão e coerência)
(Como assim? Essa ideia não está clara)
(Como assim? Reelabore essa afirrmação)
(Fale mais sobre essa situação crítica)
(Observe a falta de coesão e coerência)
(Por que eles serão afetados? Explique)
(Por que é incompatível? Explique isso)
(Por quê? Desenvolva isso em seu texto)
(Por quê? Desenvolva melhor essa ideia)
(Por quê? Fale sobre isso no seu texto)
(Qual ato? Não está explícito no texto)
(Qual seria a capacidade física ideal?)
(Que condições são essas? Exemplifique)
(Que tipo de projeto? Discuta no texto)
(Tente explicar o porquê disso ocorrer)
(inicie parágrafos com letra maiúscula)
(início de período com letra maiúscula)
(Evite esse tipo de construção no texto)
(Explique o porquê de tanta repercussão)
(Fale sobre essas rupturas no seu texto)
(Formule melhor as ideias desse período)
(Formule melhor as ideias deste período)
(Ideias confusas deste período.Revise-o)
(Por que são pontos positivos? Explique)
(Por quê? Deixe isso claro no seu texto)
(Quais? Discuta essa ideia no seu texto)
(Que tipo de desigualdade? Exemplifique)
(Que tipo de investimento? Exemplifique)
(Que tipo de investimento?Cite no texto)
(sem vírgula, acrescentar o ponto final)
(A – início de frase com letra maiúscula)
(Cuidado ao fazer esse tipo de afirmação)
(Diferente como? Deixe evidente no texto)
(No que isso influência na nova reforma?)
(Os dois termos apresentam a mesma ideia)
(Para que fim? Discuta isso no seu texto)
(Por quê? Desenvolva essa ideia no texto)
(Quais medidas? Fale sobre isso no texto)
(Quais padrões? Fale sobre eles no texto)
(Que tipo de instrução? Discuta no texto)
(Solucionar de que maneira? Exemplifique)
(Trabalhe melhor as ideias deste período)
(Como assim? Desenvolva melhor essa ideia)
(Como assim? Reformule, pois está confuso)
(Desenvolva essa ideia de modo mais claro)
(Desenvolva essa ideia; explique o porquê)
(Envidencie em seu texto o que há de ruim)
(O que seria essa "educação de qualidade?)
(Poderia – introduzir com letra maiúscula)
(Ponto final para iniciar um novo período)
(Ponto final. Aperfeiçoar de que maneira?)
(Por que estão gerando muitas discussões?)
(Por quê? Discuta essa ideia no seu texto)
(Desenvolva essa ideia de forma mais clara)
(Essa ideia não está tão evidente no texto)
(Explique o porquê disso estar acontecendo)
(Fale mais sobre essa planificação escolar)
(Fica muito vago. Cite ao menos um exemplo)
(Por quê? Deixe isso evidente em seu texto)
(Por quê? Isso não está claro no seu texto)
(Procure deixar isso evidente em seu texto)
(Use mesma quando houver sentindo de igual)
(Como assim? Desenvolva essa ideia no texto)
(Como assim? Reflita acerca dessa afirmação)
(Desenvolva essa ideia e apresente exemplos)
(O que vocês quis dizer aqui? Ficou confuso)
(Observe que há um equívoco nessa afirmação)
(Por que isso ocorre? Discuta isso no texto)
(Por quê? Apresente essa discussão no texto)
(Quais atitudes? Deixe-as evidente no texto)
(Que tipo de frustação? Cite-a no seu texto)
(Que tipo de impasse? Desenvolva essa ideia)
(Que tipo de palestra? Deixe claro no texto)
(Usa-se vírgula ao invés do ponto e vírgula)
(Apresente fatos para afirmar essa colocação)
(Isso, de fato, é tão simples assim? Reflita)
(Melhorar sob qual perspectiva? Exemplifique)
(Por que ela é bem-vinda? Apresente no texto)
(Por quê? Desenvolva essa ideia no seu texto)
(Quais são os valores morais certos? Reflita)
(Quais valores sociais? Cite-os no seu texto)
(Será que só agora isso veio à tona? Reflita)
(Desenvola essa ideia, pois ela está limitada)
(Desenvolva essa ideia, pois ela está confusa)
(Evite esse tipo de expressão clichê no texto)
(Problema na construção de sentido. Reformule)
(Quais ajustes? Isso não ficou claro no texto)
(Quais jovens? Os violentos ou os agressores?)
(Qual impasse? Não está tão evidente no texto)
(Qual é a forma de conscientizar a sociedade?)
(Que tipo de repercussão? Cite-a no seu texto)
(neste caso não é necessário o uso da vírgula)
(neste caso,não é necessário o uso de vírgula)
(usa-se "mais", pois é dada a ideia de adição)
(Cuidado ao afirmar isso. Reformule essa ideia)
(Deixe evidente o que seria essa reforma geral)
(Desenvolva essa afirmação de forma mais clara)
(E o que isso pode acarretar? Discuta no texto)
(Essa ideia não está tão evidente no seu texto)
(Por quê? Deixe claro o porquê dessa afirmação)
(Problemas na construção de sentido. Reformule)
(neste caso,não pe necessário o uso sa vírgula)
(Como assim? Essa ideia está confusa. Reformule)
(Desenvolva essa ideia, pois ela não está clara)
(Dê exemplos para reafirmar esse posicionamento)
(Explique o que seria uma reforma bem elaborada)
(Muda de que maneira? Deixe isso claro no texto)
(O que seria um comportamento "fora do normal"?)
(atenção, esta afirmação é falsa, certifique-se)
(com ressalva para os casos de real necessidade)
(Como assim? Cuidado ao fazer algumas afirmações)
(Explique o porquê dessa afirmação. Exemplifique)
(Fale sobre a estrutura que você considera ideal)
(Observe o uso de letra maiúscula após a vírgula)
(Problema na construção de sentido desse período)
(Quais? Perceba que não fora mencionado no texto)
(Qual ato? Você não apresenta o que é o bullying)
(Que tipo de incentivo? Fale sobre isso no texto)
(Melhoraria de que forma? Procure evidenciar isso)
(Por quê? Evidencie o seu ponto de vista no texto)
(Reformule essa ideia, visto que ela está confusa)
(têm dificuldades de interpretação e de expressão)
(.ponto final, início com letra maiúscula – Muitos)
(Como isso vai resolver o problema? Deixe evidente)
(Explique o porquê dessa reforma ser mal elaborada)
(O que de fato seria o diferente? Discuta no texto)
(O que está ruim? Deixe isso evidente em seu texto)
(Qual organização? Observe que ficou confuso, vago)
(Como assim? Desenvolva essa ideia com mais clareza)
(Como assim? Essa afirmação ficou estranha. Reflita)
(Como assim? Observe e reflita sobre essa afirmação)
(Como assim? Problema na construção da argumentação)
(Duas conjunções que expressam a ideia de conclusão)
(Por que isso acontece? Evidencie isso no seu texto)
(Quais? Pontue a que você achar de maior relevância)
(Você citou apenas uma consequência. Cite as demais)
(deve aparecer entre aspas, por se tratar de ironia)
(Como assim? Cuidado ao fazer esse tipo de afirmação)
(Como assim? Não ficou claro o que você queria dizer)
(Cuidado ao propor isso. Que tipo de punição severa?)
(Fale mais sobre essa ideia no seu texto; exemplique)
(Ideia que precisa ser desenvolvida com mais clareza)
(ponto final e início de período com letra maiúscula)
(Explique o porquê dessas mudanças dividirem opiniões)
(Quais? Cuidado com esse tipo de colocação. Reformule)
(Que tipo de discussões? Desenvolva isso no seu texto)
(Conscientizar de que maneira? Apresente isso no texto)
(Cuidado ao fazer essa afirmação. Reformule essa ideia)
(Fale mais sobre o que seria essa reforma no seu texto)
(Mas isso decorre de um problema psiicológico. Reflita)
(Não é tão comum assim. Reflita acerca dessa afirmação)
(Problema na construção de sentido. Reveja e reformule)
(Use pronomes relativos para evitar o queísmo no texto)
(Comece falando em que momento histórico isso aconteceu)
(Observe essa afirmação e reflita. Não ficou estranho? )
(Quais seriam essas partes significativas? Exemplifique)
(Quais são os tipos de ameaças? Deixe evidente no texto)
(Eles não foram tão discutidos no seu texto. Dê exemplos)
(Explique como o bullying se desenvolve no âmbito sexual)
(Por que estão mudando? Deixe isso evidente em seu texto)
(De que modo isso seria possível? Apresente isso no texto)
(Descreva o que seria esse cenário triste do ensino médio)
(Por que não seria bom? Apresente a sua reflexão no texto)
(Qual seria o padrão social aceito? Discuta isso no texto)
(Que tipo de construção? Isso não está claro no seu texto)
(Você tem esse tipo de problemas com alunos de sete anos?)
(O que ainda falta ser discutido? Deixe isso claro notexto)
(O que seriam melhores condições de trabalho? Exemplifique)
(Poblema na construção de sentido desse período. Reformule)
(Ponto final para iniciar um novo período; falta de coesão)
(Por quê? Deixe o seu ponto de vista evidente no seu texto)
(Problema na construção de sentido. Reformule esse período)
(Reformule, pois não ficou claro o que você deseja apontar)
(procure não utilizar essa expressão; utilize “Atualmente”)
(Como assim? Apresente evidencias que possam comprovar isso)
(Como asssim? Cuidado ao afirmar isso. Reformule essa ideia)
(Onde essa informação foi apresentada? A evidencie no texto)
(Que tipo de desigualdade? Deixe isso evidente em seu texto)
(Que tipo de melhoria? Isso não está explícito no seu texto)
(você já caracterizou no primeiro paragrágo que é no Brasil)
(Isso acontece só no Brasil? Reflita acerca dessa informação)
(Não use primeira pessoa no texto dissertativo-argumentativo)
(O que ele afirmava? Abra aspas e cite o ponto de vista dele)
(Observe que há problema na construção de sentido. Reformule)
(Problema na construção de sentido. Reelabore essa afirmação)
(Que tipo de evolução, pois, assim, a ideia fica muito ampla)
(Que tipo de ferramenta? Deixe essa ideia clara em seu texto)
(Usa-se "mesmo" quando se quer atribuir sentido de igualdade)
(De que maneira? Discuta isso no texto. Desenvolva essa ideia)
(Observe que há problemas na construção de sentido. Reformule)
(Problema na construção de sentido. Reveja coesão e coerência)
(Quando isso foi dito? Foi dito em que veículo de informação?)
(Reflita essa afirmação; busque o significado de meritocrácia)
(Reflita sobre essa afirmação. Esse realmente seria o motivo?)
(quais filosofos? Cite pelo um para comprovar seus argumentos)
(Depois de ponto final inicia-se o período com letra maiúscula)
(tome cuidado ao utilizar esta palavra, certifique-se primeiro)
(Apresente fatos que comprovem os acontecimentos que você citou)
(Não compreende a estrutura do texto dissertativo-argumentativo)
(Boa argumentação, mas precisa ser desenvolvida com mais clareza)
(Como assim? Desenvolva melhor essa ideia, pois ela está confusa)
(Cuidado com algumas afirmações. Reveja essa ideia e a reelabore)
(Cuidado com esse tipo afirmação. Lembre-se dos Direitos Humanos)
(De que maneira? Deixe esse posicionamento evidente em seu texto)
(Desenvolva essa ideia com mais clareza. Deixe evidente no texto)
(Desenvolva essa ideia para que fique claro o seu posicionamento)
(Fale sobre a atual situação da educação brasileira no seu texto)
(Ideias confusas. Elabore-as com mais clareza. Exemplifique mais)
(Por que esses países são exemplos? Desenvolva isso no seu texto)
(Apresente um fato que evidencie o porquê disso estar acontecendo)
(É preciso apresentar mais argumentos para que isso fique evidente)
(Em algum momento isso foi possível? Reflita acerca dessa afirmação)
(Explique o porquê diss pois essa ideia não está clara em seu texto)
(Isso acontece só com os alunos? Reflita e reformule essa colocação)
(Notado de que maneira? Deixe isso claro no seu texto. Exemplifique)
(O que poderia sanar os problemas? Deixe isso evidente no seu texto)
(Que tipo de melhoria? O que está ruim? Evidencie isso no seu texto)
(Redundância/você já usou estas mesmas ideias no paragráfo anterior)
(responda à essa pergunta no máximo no inpicio do proximo parágrafo)
(Como seriam as mudanças bem formuladas? Fale sobre isso no seu texto)
(Por que isso é importante? Seria interessante discutir isso no texto)
(Quando? Apresente esse fato e explique o porquê disso ter acontecido)
(É interessante mencionar dados para dar validade a essa argumentação)
(Como assim? Reflita e reelabore essa afirmação, pois ela está confusa)
(Cuidado ao afirmar isso, visto que a reforma entrará em vigor em 2018)
(Elas não estão preparadas de que maneira?Fale sobre isso no seu texto)
(Que ideial é esse? Apresente no texto, pois essa informação está vaga)
(Como assim? Reflita sobre essa informação. De fato, isso não acontece?)
(Explique o porquê disso estar acontecendo. Evidencie isso no seu texto)
(O que seria uma educação mais igualitária? Evidencie isso no seu texto)
(Problema na construção de sentido. Essa afirmação está confusa. Reflita)
(Qual séria a causa desse resultado? Procure trabalhar isso no seu texto)
(Vale lembrar que o âmbito escolar não se resume em professores e alunos)
(Apresenta o poblema da infraestrutura, mas não articula para uma solução)
(Desenvolva essa ideia e apresente exemplos que justifiquem o porquê disso)
(O que seria ensino clássico e contemporâneo? Fale sobre isso no seu texto)
(Dê exemplos. Desenvolva mais essa ideia nesse parágrafo de desenvolvimento)
(Por que ela pode ser relacionada ao Fordismo? Deixe isso evidente no texto)
(Veja alguns elementos coesivos que podem melhorar a coerência no seu texto)
(Como assim? Reformule essa ideia. Qual seria o problema em questão? Reflita)
(De certa forma, o apelido também pode ser uma forma de preconceito. Reflita)
(Isso não fica evidente no texto. Reelabore a sua argumentação expondo fatos)
(Os atos verbais não são violentos? Relflita e reformule, pois ficou confuso)
(Dispensável, pois ela está no mesmo campo semântico de humilhação e opressão)
(Observe que não há um direcionamento de quando é o fim da citação. Reveja isso)
(período confuso – a criminalidade é um deles, “um deles” o que? Quem são eles?)
(Como assim? Apresente exemplos acerca do fato para dar validade a sua afirmação)
(De que maneira? Dê um exemplo do que pode ser feito para reverter essa situação)
(Explique o porquê dessas matérias serem essenciais para o conhecimento de mundo)
(Como assim? Observe que há um erro na construção de sentido. Reformule essa ideia)
(Qual seria o modelo de educação ideal para o Brasil? Fale sobre isso em seu texto)
(como por exemplo, decapitações de jornalistas, bombardeios e entre outros métodos)
(use letra maíscula para se referir a determinada Instituição anteriormente citada)
(Como a reforma poderia tirar o conhecimento do aluno? Desenvolva melhor essa ideia)
(Explique a importância dessas disciplinas no seu texto e que disciplinas são essas)
(Observe que há uma diferença muito grande entre escola pública e escola particular)
(Cuidado ao afirmar isso, pois pode ferir os direitos humanos. Punir em que sentido?)
(Observe que essa informação não apresenta coerência com o que foi discutido anteriormente)
(Observe que o MEC e o Ministério da educação são a mesma coisa, o que muda é a abreviação)
(Eliminar de que maneira? Cuidado ao afirmar isso, pois vai de encontro aos direitos humanos)
(Qual é o motivo do surgimento desses problemas? Deixe essa informação evidente no seu texto)
(Cuidado ao fazer essa afirmação. Apresente fatos que encaminhem as discussões acerca do tema)
(Desenvolva essa ideia no seu texto para deixar a sua argumentação clara e apresente exemplos)
(Qual dos problemas citados? A evasão escolar ou o desinteresse por determinadas disciplinas?)
(Que tipo de melhoria?,(Ponto final para o início de um novo perído; problemas de coesão textual)
(como a retirada de um assunto que demonstre, por exemplo, a má utilização dos recursos públicos)
(Qual seria a possível causa disso? É interessante que haja uma discussão sobre isso no seu texto)
(Como assim? As ideias aqui presentes estão confusas. Cuidado ao fazer algumas afirmações. Reflita)
(Reflita sobre esse tipo de construção, visto que a escola faz "parte do mundo". Reformule essa ideia)
(Como assim? Observe que há um equívoco quando você afirma isso, visto que o governo não ignora o Enem)
(Observe que há uma generalização aqui, pois nem todos fazer curso técnico após terminar o ensino médio)
(Quais disciplinas não serão obrigatórias? Fale sobre elas no seu texto e evidencie um possível problema)
(Observe essa afirmação e procure relacionar a realidade do nosso país com a realidade social da Finlância)
(Essa ideia não está tão evidente no seu texto. Reformule a sua linha de argumentação para que isso fique claro)
(Como assim? Reformule, tendo em vista que é contraditório de acordo com o que você menciona no período anterior)
(Observe que todos os ambientes citados por você fazem parte de um meio social. Logo, essa colocação é dispensável)
(argumento sem fundamento, pois o Papa João Paulo II permaneceu no pontificado até sua morte em 2 de abril de 2005)
(Quais? Observe que esse tipo de construção feita nesse período é para evocar uma ideia que fora mencionada anteriormente)
(Facilitado de que maneira? Cuidado ao fazer certas generalizações. O conhecimento está, de fato, facilitado a todos? Reflita)
(Como assim? Cuidado ao fazer essee tipo de colocação. Realmente é possível dizer que uma pessoa é mais inteligente que a outra? Reflita)
""".split('\n'))



In [ ]:
padroes_correcoes = """( "por isso" entre vírgulas)
( acento agudo)
( o "os" soa melhor e pode substituir o verbo possuem)
((;)
((Sem vírgula)
((Vírgula)
(***)
(, )
(, de acordo)
(, em)
(, entre outras)
(, já)
(, o que implica)
(, o que)
(, por exemplo)
(, uma vez que)
(,)
(,como dizia)
(,entregues pelo mordomo pessoal,)
(,que)
(-)
(-os)
(-se)
(. ponto final)
(.)
(...)
(.ponto final)
(.ponto final, início com letra maiúscula – Muitos)
(1)
(2)
(3)
(4)
(:)
(;)
(?)
(??)
(???)
(A campanha)
(A experiência do)
(A frase)
(A guerra invisível)
(A internet é)
(A notícia de)
(A partir)
(A reforma aumentará a carga horária. Observe que existe um equívoco nessa afirmação)
(A reforma)
(A – início de frase com letra maiúscula)
(A)
(Acabou)
(Afeta de que maneira?)
(Afetar em que sentido?)
(Afetaria de que maneira?)
(Afinal,)
(Afirmação equivocada)
(Agora, frente à)
(Ainda)
(Algumas)
(Alguns deles quem?)
(Alguns)
(Além)
(Ambos)
(Amenizadas de que maneira?)
(Andes)
(Ao)
(Apenas punir)
(Apresenta ideais confusas)
(Apresenta o poblema da infraestrutura, mas não articula para uma solução)
(Apresente esse fato)
(Apresente esse índice para sustentar o seu argumento)
(Apresente fatos para afirmar essa colocação)
(Apresente fatos que comprovem os acontecimentos que você citou)
(Apresente pontos em comum)
(Apresente um fato que evidencie o porquê disso estar acontecendo)
(Assim)
(Até)
(Boa argumentação, mas precisa ser desenvolvida com mais clareza)
(C )
(Cantareira, )
(Católica)
(Cinco Olhos)
(Cite alguns exemplos)
(Cite alguns)
(Cite esses países no seu texto)
(Cite exemplos do que seria ideal)
(Cite exemplos)
(Cite o nome)
(Cite uma fonte)
(Cite)
(Cite-as)
(Cite-os)
(Clichê)
(Coloque entre aspas)
(Com – letra maiúscula)
(Comece falando em que momento histórico isso aconteceu)
(Como a reforma poderia tirar o conhecimento do aluno? Desenvolva melhor essa ideia)
(Como assim? A ideia está confusa)
(Como assim? Apresente evidencias que possam comprovar isso)
(Como assim? Apresente exemplos acerca do fato para dar validade a sua afirmação)
(Como assim? As ideias aqui presentes estão confusas. Cuidado ao fazer algumas afirmações. Reflita)
(Como assim? Cuidado ao fazer algumas afirmações)
(Como assim? Cuidado ao fazer esse tipo de afirmação)
(Como assim? Cuidado ao fazer essee tipo de colocação. Realmente é possível dizer que uma pessoa é mais inteligente que a outra? Reflita)
(Como assim? Desenvolva essa ideia com mais clareza)
(Como assim? Desenvolva essa ideia no texto)
(Como assim? Desenvolva essa ideia)
(Como assim? Desenvolva melhor essa ideia)
(Como assim? Desenvolva melhor essa ideia, pois ela está confusa)
(Como assim? Discuta no texto)
(Como assim? Essa afirmação ficou estranha. Reflita)
(Como assim? Essa ideia está confusa. Reformule)
(Como assim? Essa ideia não está clara)
(Como assim? Estamos no século XXI)
(Como assim? Evidencie no texto)
(Como assim? Ficou confuso)
(Como assim? Não ficou claro o que você queria dizer)
(Como assim? Não ficou claro)
(Como assim? Observe e reflita sobre essa afirmação)
(Como assim? Observe que há um equívoco quando você afirma isso, visto que o governo não ignora o Enem)
(Como assim? Observe que há um erro na construção de sentido. Reformule essa ideia)
(Como assim? Problema na construção da argumentação)
(Como assim? Quais?)
(Como assim? Reelabore essa afirrmação)
(Como assim? Reflita acerca dessa afirmação)
(Como assim? Reflita e reelabore essa afirmação, pois ela está confusa)
(Como assim? Reflita sobre essa informação. De fato, isso não acontece?)
(Como assim? Reformule essa afirmação)
(Como assim? Reformule essa ideia)
(Como assim? Reformule essa ideia. Qual seria o problema em questão? Reflita)
(Como assim? Reformule)
(Como assim? Reformule, pois está confuso)
(Como assim? Reformule, tendo em vista que é contraditório de acordo com o que você menciona no período anterior)
(Como assim?)
(Como asssim? Cuidado ao afirmar isso. Reformule essa ideia)
(Como asssim? Reformule essa ideia)
(Como isso vai resolver o problema? Deixe evidente)
(Como no)
(Como seriam as mudanças bem formuladas? Fale sobre isso no seu texto)
(Como)
(Como? Apresente no texto)
(Confuso)
(Conscientizar de que maneira? Apresente isso no texto)
(Conscientizar de que maneira?)
(Conselho)
(Cotas para negros)
(Cuidado ao afirmar isso)
(Cuidado ao afirmar isso, pois pode ferir os direitos humanos. Punir em que sentido?)
(Cuidado ao afirmar isso, visto que a reforma entrará em vigor em 2018)
(Cuidado ao afirmar isso, visto que vai de encontro aos direitos humanos)
(Cuidado ao afirmar isso. Reformule essa ideia)
(Cuidado ao fazer algumas afirmações)
(Cuidado ao fazer certas afirmações)
(Cuidado ao fazer essa afirmação)
(Cuidado ao fazer essa afirmação. Apresente fatos que encaminhem as discussões acerca do tema)
(Cuidado ao fazer essa afirmação. Reformule essa ideia)
(Cuidado ao fazer esse tipo de afirmação)
(Cuidado ao fazer generaliações)
(Cuidado ao fazer generalizações)
(Cuidado ao generalizar)
(Cuidado ao generalizar. Reflita acerca dessa informação)
(Cuidado ao propor isso. Que tipo de punição severa?)
(Cuidado com algumas afirmações)
(Cuidado com algumas afirmações. Reveja essa ideia e a reelabore)
(Cuidado com esse termo "invadir")
(Cuidado com esse tipo afirmação. Lembre-se dos Direitos Humanos)
(Cuidado com esse tipo de afirmação)
(De certa forma, o apelido também pode ser uma forma de preconceito. Reflita)
(De que forma isso acontece?)
(De que maneira isso seria possível)
(De que maneira? Deixe esse posicionamento evidente em seu texto)
(De que maneira? Discuta isso no texto. Desenvolva essa ideia)
(De que maneira? Dê um exemplo do que pode ser feito para reverter essa situação)
(De que maneira?)
(De que modo isso seria possível? Apresente isso no texto)
(De que modo)
(De que modo?)
(Deixe evidente o que seria essa reforma geral)
(Deixe o motivo evidente no seu texto)
(Denúncias)
(Depois de ponto final inicia-se o período com letra maiúscula)
(Descreva essa situação em seu texto)
(Descreva o que seria esse cenário triste do ensino médio)
(Desenvola essa ideia, pois ela está limitada)
(Desenvolva as ideias desse parágrafo)
(Desenvolva essa afirmação de forma mais clara)
(Desenvolva essa afirmação)
(Desenvolva essa ideia com mais clareza. Deixe evidente no texto)
(Desenvolva essa ideia de forma mais clara)
(Desenvolva essa ideia de modo mais claro)
(Desenvolva essa ideia e apresente exemplos que justifiquem o porquê disso)
(Desenvolva essa ideia e apresente exemplos)
(Desenvolva essa ideia e exemplifique)
(Desenvolva essa ideia no seu texto para deixar a sua argumentação clara e apresente exemplos)
(Desenvolva essa ideia no texto)
(Desenvolva essa ideia para que fique claro o seu posicionamento)
(Desenvolva essa ideia)
(Desenvolva essa ideia, pois ela está confusa)
(Desenvolva essa ideia, pois ela não está clara)
(Desenvolva essa ideia; explique o porquê)
(Desenvolva mais essa ideia)
(Desenvolva melhor essa ideia)
(Desenvolva)
(Desse modo)
(Deste modo)
(Deve ainda)
(Deveria ou deve)
(Deveria ou deve? Reflita)
(Dia – inicie com letra maiúscula)
(Diante d)
(Diante de)
(Diferente como? Deixe evidente no texto)
(Discussão limitada)
(Discuta essa ideia no texto)
(Discutida por quem? Exemplifique)
(Dispensável, pois ela está no mesmo campo semântico de humilhação e opressão)
(Ditadura Militar)
(Dois pontos)
(Duas conjunções que expressam a ideia de conclusão)
(Durkheim)
(Dê exemplo)
(Dê exemplos de mudanças)
(Dê exemplos dessas agressões)
(Dê exemplos dessas situações)
(Dê exemplos desse tipo de agressão)
(Dê exemplos desses atos violentos)
(Dê exemplos para reafirmar esse posicionamento)
(Dê exemplos)
(Dê exemplos. Desenvolva mais essa ideia nesse parágrafo de desenvolvimento)
(Dê um exemplo disso)
(Dê um exemplo)
(E o que isso pode acarretar? Discuta no texto)
(Ela)
(Elas não estão preparadas de que maneira?Fale sobre isso no seu texto)
(Ele, – com letra maiúscula e vírgula)
(Eles não foram tão discutidos no seu texto. Dê exemplos)
(Eliminar de que maneira? Cuidado ao afirmar isso, pois vai de encontro aos direitos humanos)
(Em algum momento isso foi possível? Reflita acerca dessa afirmação)
(Em relação à)
(Em)
(Enfrente)
(Entre aspas pois é uma citação)
(Entre vírgulas)
(Envidencie em seu texto o que há de ruim)
(Eros)
(Especulações)
(Essa ideia não está tão evidente no seu texto)
(Essa ideia não está tão evidente no seu texto. Reformule a sua linha de argumentação para que isso fique claro)
(Essa ideia não está tão evidente no texto)
(Essas são)
(Esse)
(Esses)
(Esta renúncia)
(Esta)
(Estado)
(Este é)
(Evidencie isso por meio de fatos)
(Evite esse tipo de construção no texto)
(Evite esse tipo de expressão clichê no texto)
(Evite o piegas no texto)
(Executivo)
(Exemplifique e discuta as causas)
(Exemplifique)
(Exemplifique; ponto final)
(Explique a importância dessas disciplinas no seu texto e que disciplinas são essas)
(Explique a razão disso)
(Explique como o bullying se desenvolve no âmbito sexual)
(Explique melhor essa ideia)
(Explique meslhor essa afirmação)
(Explique o porquê de tal afirmação)
(Explique o porquê de tanta repercussão)
(Explique o porquê de ter dado certo)
(Explique o porquê dessa afirmação)
(Explique o porquê dessa afirmação. Exemplifique)
(Explique o porquê dessa reforma ser mal elaborada)
(Explique o porquê dessas matérias serem essenciais para o conhecimento de mundo)
(Explique o porquê dessas mudanças dividirem opiniões)
(Explique o porquê diss pois essa ideia não está clara em seu texto)
(Explique o porquê disso estar acontecendo)
(Explique o porquê disso estar acontecendo. Evidencie isso no seu texto)
(Explique o porquê disso ocorrer)
(Explique o porquê disso)
(Explique o porquê)
(Explique o que seria uma reforma bem elaborada)
(Explique que reforma seria essa)
(Expressão clichê)
(Expressão clichê. Sem vírgula)
(FEBEM’s – diga o que significa a sigla)
(Facilitado de que maneira? Cuidado ao fazer certas generalizações. O conhecimento está, de fato, facilitado a todos? Reflita)
(Fale mais sobre essa ideia no seu texto; exemplique)
(Fale mais sobre essa planificação escolar)
(Fale mais sobre essa situação crítica)
(Fale mais sobre esse acordo)
(Fale mais sobre isso no seu texto)
(Fale mais sobre isso;)
(Fale mais sobre o que seria essa reforma no seu texto)
(Fale sobre a atual situação da educação brasileira no seu texto)
(Fale sobre a estrutura que você considera ideal)
(Fale sobre a proposta)
(Fale sobre essas mudanças)
(Fale sobre essas rupturas no seu texto)
(Fale sobre isso no seu texto)
(Fale sobre o porquê dessa discussão)
(Fale um pouco mais desses padrões)
(Falhar em que sentido?)
(Falou o quê?)
(Falta de conhecimento sobre o quê?)
(Famosos)
(Fica muito vago. Cite ao menos um exemplo)
(Foram)
(Formule melhor as ideias desse período)
(Formule melhor as ideias deste período)
(Fraternidade)
(Gregório)
(Houve um)
(Houve)
(Há no Brasil)
(Há também)
(Há uma lei que trate disso)
(Há)
(Ideia clichê)
(Ideia que precisa ser desenvolvida com mais clareza)
(Ideias confusas deste período.Revise-o)
(Ideias confusas)
(Ideias confusas. Elabore-as com mais clareza. Exemplifique mais)
(Igreja)
(Igualdade)
(Iluminismo)
(Incentivar de que maneira?)
(Informação dispensável)
(Iniciação de paragráfo sempre o uso de letra maiúscula)
(Iniciação de paragráfo)
(Instituição)
(Intimidar de que maneira?)
(Isso acontece só com os alunos? Reflita e reformule essa colocação)
(Isso acontece só no Brasil? Reflita acerca dessa informação)
(Isso não está claro no seu texto)
(Isso não está explícito no texto)
(Isso não está tão claro no texto)
(Isso não está tão evidente no texto)
(Isso não fica evidente no texto. Reelabore a sua argumentação expondo fatos)
(Isso não ocorrerá em todos os casos)
(Isso o quê?)
(Isso pode ser feito)
(Isso é reflexo da)
(Isso)
(Isso, de fato, é tão simples assim? Reflita)
(Lei da)
(Letra maiúscula em início de frase)
(Letra maiúscula)
(Letra maiúsula)
(Letra minúscula)
(Líder)
(Marca de oralidade)
(Marcas de oralidade)
(Mas isso decorre de um problema psiicológico. Reflita)
(Mas isso está previsto na reforma)
(Mas o “por que”)
(Mas)
(MedioTec)
(Melhorar sob qual perspectiva? Exemplifique)
(Melhoraria de que forma? Procure evidenciar isso)
(Movimento dos sem terras)
(Muda de que maneira? Deixe isso claro no texto)
(Muitas das)
(Muito se)
(Muitos quem? Por quê?)
(Muitos quem?)
(Muitos)
(Nem todos quem?)
(Networking)
(No Brasil)
(No Brasil,muitos jovens são)
(No dia)
(No que isso influência na nova reforma?)
(No)
(Nos)
(Notado de que maneira? Deixe isso claro no seu texto. Exemplifique)
(Não compreende a estrutura do texto dissertativo-argumentativo)
(Não está evidente no seu texto)
(Não está explícito no texto)
(Não ficou claro. Refaça esse período)
(Não há "deste")
(Não use primeira pessoa no texto dissertativo-argumentativo)
(Não é necessário)
(Não é tão comum assim. Reflita acerca dessa afirmação)
(Não – com letra maiúscula)
(O Uruguai)
(O acesso)
(O filósofo)
(O fim dos problema)
(O governo deve criar)
(O primeiro caso)
(O que ainda falta ser discutido? Deixe isso claro notexto)
(O que de fato seria o diferente? Discuta no texto)
(O que ele afirmava? Abra aspas e cite o ponto de vista dele)
(O que está ruim? Deixe isso evidente em seu texto)
(O que poderia sanar os problemas? Deixe isso evidente no seu texto)
(O que seria ensino clássico e contemporâneo? Fale sobre isso no seu texto)
(O que seria essa "educação de qualidade?)
(O que seria essa verdadeira mudança?)
(O que seria um comportamento "fora do normal"?)
(O que seria uma conversa relevante?)
(O que seria uma educação mais igualitária? Evidencie isso no seu texto)
(O que seria uma sociedade evoluída?)
(O que seriam melhores condições de trabalho? Exemplifique)
(O que são coisas?)
(O que você quis dizer?)
(O que vocês quis dizer aqui? Ficou confuso)
(O que é coisa?)
(O quê? Evidencie isso no seu texto)
(O quê?)
(O vírus foi)
(O – maiúsculo)
(O)
(Observando)
(Observe a falta de coesão e coerência)
(Observe essa afirmação e procure relacionar a realidade do nosso país com a realidade social da Finlância)
(Observe essa afirmação e reflita. Não ficou estranho? )
(Observe essa afirmação)
(Observe o uso de letra maiúscula após a vírgula)
(Observe que essa informação não apresenta coerência com o que foi discutido anteriormente)
(Observe que há problema na construção de sentido. Reformule)
(Observe que há problemas na construção de sentido. Reformule)
(Observe que há um equívoco nessa afirmação)
(Observe que há uma diferença muito grande entre escola pública e escola particular)
(Observe que há uma generalização aqui, pois nem todos fazer curso técnico após terminar o ensino médio)
(Observe que não há um direcionamento de quando é o fim da citação. Reveja isso)
(Observe que o Enem foi feito pelos alunos um mês após a data marcada para o acontecimento do exame)
(Observe que o MEC e o Ministério da educação são a mesma coisa, o que muda é a abreviação)
(Observe que todos os ambientes citados por você fazem parte de um meio social. Logo, essa colocação é dispensável)
(Onde essa informação foi apresentada? A evidencie no texto)
(Onde isso ocorreu? Dê exemplos)
(Os apoiadores)
(Os atos verbais não são violentos? Relflita e reformule, pois ficou confuso)
(Os dois termos apresentam a mesma ideia)
(Os outros quem?)
(Outro)
(Outros quem?)
(Outros)
(Para que fim? Discuta isso no seu texto)
(Paragráfo)
(Parágrafo confuso. Reformule)
(Parágrafo sem coerência)
(Passam)
(Pena de Morte)
(Perseguir o quê?)
(Período com ideias confusas.Revise)
(Período confuso)
(Período muito extenso)
(Pesquisa realizada por quem?)
(Piegas)
(Piorando em que sentido? Deixe isso evidente no seu texto)
(Pleonasmo)
(Poblema na construção de sentido desse período. Reformule)
(Poderia – introduzir com letra maiúscula)
(Ponto fina)
(Ponto final para iniciar um novo período)
(Ponto final para iniciar um novo período; falta de coesão)
(Ponto final)
(Ponto final. Aperfeiçoar de que maneira?)
(Ponto fnal)
(Por que ela pode ser relacionada ao Fordismo? Deixe isso evidente no texto)
(Por que ela é bem-vinda? Apresente no texto)
(Por que ele é defasado? Explique)
(Por que eles serão afetados? Explique)
(Por que esses países são exemplos? Desenvolva isso no seu texto)
(Por que estão gerando muitas discussões?)
(Por que estão mudando? Deixe isso evidente em seu texto)
(Por que isso acontece? Evidencie isso no seu texto)
(Por que isso ocorre? Discuta isso no texto)
(Por que isso ocorre?)
(Por que isso é importante? Seria interessante discutir isso no texto)
(Por que não seria bom? Apresente a sua reflexão no texto)
(Por que são pontos positivos? Explique)
(Por que é incompatível? Explique isso)
(Por quem? Cite no texto)
(Por quem?)
(Por quê? Apresente essa discussão no texto)
(Por quê? Cite uma causa disso)
(Por quê? Deixe claro no seu texto)
(Por quê? Deixe claro o porquê dessa afirmação)
(Por quê? Deixe evidente no texto)
(Por quê? Deixe isso claro no seu texto)
(Por quê? Deixe isso evidente em seu texto)
(Por quê? Deixe o seu ponto de vista evidente no seu texto)
(Por quê? Desenvolva essa discussão)
(Por quê? Desenvolva essa ideia no seu texto)
(Por quê? Desenvolva essa ideia no texto)
(Por quê? Desenvolva essa ideia)
(Por quê? Desenvolva isso em seu texto)
(Por quê? Desenvolva melhor essa ideia)
(Por quê? Discuta essa ideia no seu texto)
(Por quê? Discuta essa ideia no texto)
(Por quê? Discuta essa ideia)
(Por quê? Discuta isso no seu texto)
(Por quê? Discuta isso no texto)
(Por quê? Evidencie isso no seu texto)
(Por quê? Evidencie no texto)
(Por quê? Evidencie o motivo no texto)
(Por quê? Evidencie o seu ponto de vista no texto)
(Por quê? Exemplifique)
(Por quê? Explique isso no seu texto)
(Por quê? Explique isso no texto)
(Por quê? Explique no texto)
(Por quê? Explique)
(Por quê? Fale sobre isso no seu texto)
(Por quê? Isso não está claro no seu texto)
(Por quê? Justifique)
(Por quê?)
(Portanto ou Por entanto)
(Portanto/ Por entanto)
(Porém)
(Presidente)
(Problema de coerência)
(Problema de coesão e coerência)
(Problema na construção de sentido desse período)
(Problema na construção de sentido)
(Problema na construção de sentido. Essa afirmação está confusa. Reflita)
(Problema na construção de sentido. Reelabore essa afirmação)
(Problema na construção de sentido. Reformule esse período)
(Problema na construção de sentido. Reformule)
(Problema na construção de sentido. Reveja coesão e coerência)
(Problema na construção de sentido. Reveja e reformule)
(Problema na contrução de sentido)
(Problemas de coesão e coerência)
(Problemas na construção de sentido)
(Problemas na construção de sentido. Reformule)
(Procure deixar isso evidente em seu texto)
(Procure usar sinônimos)
(Pão e Circo)
(Quais agressões?)
(Quais ajustes? Isso não ficou claro no texto)
(Quais atitudes? Deixe-as evidente no texto)
(Quais avaliações nacionais? Cite-as)
(Quais avaliações nacionais?)
(Quais caminhos?)
(Quais consequências?)
(Quais conteúdos importantes? Cite-os)
(Quais crianças?)
(Quais diferenças? Cite-as)
(Quais disciplinas não serão obrigatórias? Fale sobre elas no seu texto e evidencie um possível problema)
(Quais especialistas? Exemplifique)
(Quais expectativas? Cite-as)
(Quais falhas? Cite exemplos)
(Quais grupos?)
(Quais humilhações? Discuta no texto)
(Quais investimentos? Dê exemplos)
(Quais jovens? Os violentos ou os agressores?)
(Quais matérias? Cite-as)
(Quais mecanismos?)
(Quais medidas? Fale sobre isso no texto)
(Quais medidas?)
(Quais melhorias? Cite-as)
(Quais mudanças? Exemplifique)
(Quais mudanças?)
(Quais obstáculos?)
(Quais padrões? Cite-os no texto)
(Quais padrões? Fale sobre eles no texto)
(Quais países? Cite-os)
(Quais pessoas?)
(Quais pontos fundamentais?)
(Quais preocupações?)
(Quais princípios?)
(Quais problemas crônicos?)
(Quais problemas? Cite-os)
(Quais projetos?)
(Quais provas?)
(Quais questões estruturais? Cite-as)
(Quais recursos?)
(Quais seriam as decisões acertadas?)
(Quais seriam essas partes significativas? Exemplifique)
(Quais seriam? Apresente no seu texto)
(Quais seriam? Cite-os)
(Quais seriam? Dê exemplos)
(Quais seriam?)
(Quais são os tipos de ameaças? Deixe evidente no texto)
(Quais são os valores morais certos? Reflita)
(Quais valores positivos? Dê exemplos)
(Quais valores sociais? Cite-os no seu texto)
(Quais valores? Dê exemplos)
(Quais valores?)
(Quais? Assim fica muito vago)
(Quais? Cite alguns exmplos)
(Quais? Cite exemplos)
(Quais? Cite no texto)
(Quais? Cite)
(Quais? Cite-as e explique o porquê)
(Quais? Cite-as)
(Quais? Cite-os no seu texto)
(Quais? Cite-os no texto)
(Quais? Cite-os)
(Quais? Como assim? Esta confuso)
(Quais? Cuidado com esse tipo de colocação. Reformule)
(Quais? Discuta essa ideia no seu texto)
(Quais? Discuta isso em seu texto)
(Quais? Discuta no texto)
(Quais? Dê exemplos)
(Quais? Exemplifique)
(Quais? Não foi citado no texto)
(Quais? Observe que esse tipo de construção feita nesse período é para evocar uma ideia que fora mencionada anteriormente)
(Quais? Perceba que não fora mencionado no texto)
(Quais? Pontue a que você achar de maior relevância)
(Quais? Você não citou no texto)
(Quais?)
(Qual ato? Não está explícito no texto)
(Qual ato? Você não apresenta o que é o bullying)
(Qual crise?)
(Qual dano histórico?)
(Qual dos problemas citados? A evasão escolar ou o desinteresse por determinadas disciplinas?)
(Qual função?)
(Qual humilhação? Dê exemplos)
(Qual ideia)
(Qual impasse? Não está tão evidente no texto)
(Qual impasse?)
(Qual modelo? Fale sobre ele)
(Qual modelo?)
(Qual opção nova?)
(Qual organização? Observe que ficou confuso, vago)
(Qual pesquisa? Apresente no texto)
(Qual proposta?)
(Qual prática?)
(Qual raiz?)
(Qual seria a capacidade física ideal?)
(Qual seria a maneira correta?)
(Qual seria a possível causa disso? É interessante que haja uma discussão sobre isso no seu texto)
(Qual seria o modelo de educação ideal para o Brasil? Fale sobre isso em seu texto)
(Qual seria o padrão social aceito? Discuta isso no texto)
(Qual séria a causa desse resultado? Procure trabalhar isso no seu texto)
(Qual é a forma de conscientizar a sociedade?)
(Qual é o motivo do surgimento desses problemas? Deixe essa informação evidente no seu texto)
(Qual)
(Qual? A primeira ou a segunda?)
(Qual? Deixe essa ideia evidente)
(Qual? Dê exemplo)
(Qual?)
(Quando isso foi dito? Foi dito em que veículo de informação?)
(Quando? Apresente esse fato e explique o porquê disso ter acontecido)
(Que condições são essas? Exemplifique)
(Que consequência futura?)
(Que efeito contrário?)
(Que ideial é esse? Apresente no texto, pois essa informação está vaga)
(Que lei é essa? Quando foi aprovada?)
(Que lei?)
(Que medidas seriam essas?)
(Que mudanças precisam ser feitas?)
(Que padrão? Discuta no texto)
(Que padrões seriam esses?)
(Que pesquisa?)
(Que pessoas seriam?)
(Que seriam? Deixe evidente no texto)
(Que seriam?)
(Que tiipo de melhorias?)
(Que tipo de construção? Isso não está claro no seu texto)
(Que tipo de desigualdade? Deixe isso evidente em seu texto)
(Que tipo de desigualdade? Exemplifique)
(Que tipo de discussões? Desenvolva isso no seu texto)
(Que tipo de evolução, pois, assim, a ideia fica muito ampla)
(Que tipo de ferramenta? Deixe essa ideia clara em seu texto)
(Que tipo de frustação? Cite-a no seu texto)
(Que tipo de impasse? Desenvolva essa ideia)
(Que tipo de incentivo? Exemplifique)
(Que tipo de incentivo? Fale sobre isso no texto)
(Que tipo de instrução? Discuta no texto)
(Que tipo de investimento)
(Que tipo de investimento? Exemplifique)
(Que tipo de investimento?)
(Que tipo de investimento?Cite no texto)
(Que tipo de melhoria? Isso não está explícito no seu texto)
(Que tipo de melhoria? O que está ruim? Evidencie isso no seu texto)
(Que tipo de melhoria?,(Ponto final para o início de um novo perído; problemas de coesão textual)
(Que tipo de mudanças?)
(Que tipo de palestra? Deixe claro no texto)
(Que tipo de projeto? Discuta no texto)
(Que tipo de punição)
(Que tipo de punições?)
(Que tipo de repercussão? Cite-a no seu texto)
(Que tipo de reunião?)
(Que tipo de solução?)
(Que trabalho seria esse?)
(Que valores são esses?)
(Que vídeo? Deixe explícito)
(Que é?)
(Quem são esses indivíduos?)
(Quem são esses muitos?)
(Quem são os maiores interessados?)
(Quem é Hobss?)
(Quem? Cite-os)
(Quem?)
(Ratzinger – acrescentar o sobrenome)
(Recentemente)
(Redundância)
(Redundância/você já usou estas mesmas ideias no paragráfo anterior)
(Reelabore essa afirmação)
(Reflita acerca dessa afirmação)
(Reflita essa afirmação; busque o significado de meritocrácia)
(Reflita sobre essa afirmação)
(Reflita sobre essa afirmação. Esse realmente seria o motivo?)
(Reflita sobre essa colocação)
(Reflita sobre esse tipo de construção, visto que a escola faz "parte do mundo". Reformule essa ideia)
(Reflita sobre isso)
(Reflitta sobre essa afirmação)
(Reformule essa ideia)
(Reformule essa ideia, visto que ela está confusa)
(Reformule esse período)
(Reformule)
(Reformule, pois não ficou claro o que você deseja apontar)
(Reformule. Reveja coesão e coerência)
(Renúncia)
(Repense acerca dessa afirmação)
(Representa a mesma ideia do que é dito anteriormente. Desse modo, essa indormação é dispensável ao texto)
(Reveja essa afirmação)
(Rousseff)
(Se vírgula)
(Se)
(Sem ":")
(Sem "se")
(Sem aspas)
(Sem dois pontos)
(Sem dois pontos; letra minúscula)
(Sem hífen)
(Sem os dois pontos)
(Sem parênteses)
(Sem ponto de exclamação)
(Sem ponto e vírgula)
(Sem ponto final)
(Sem ponto final, apenas a vírgula)
(Sem ponto final, usa-se vírgula)
(Sem ponto final. Usa-se vírgula)
(Sem ponto final. Use a vírgula)
(Sem travessão)
(Sem vírgla)
(Sem vírgul)
(Sem vírgula)
(Sem vírgula. Usa-se ponto final)
(Sem vírgulas)
(Sem vírgulas. Usa-se dois pontos)
(Sendo assim)
(Senso comum)
(Será que só agora isso veio à tona? Reflita)
(Será)
(Sinafese)
(Socialismo)
(Solucionar de que maneira? Exemplifique)
(Somente o ponto final)
(São exemplos)
(São)
(Só a lei é suficiente?)
(Também deve-se investir em)
(Também)
(Tem-se falado)
(Temas)
(Tente explicar o porquê disso ocorrer)
(Tipo de argumento clichê)
(Tipo de construção clichê)
(Tipo de construção piegas)
(Todavia)
(Todos pela Educação)
(Trabalhe melhor as ideias deste período)
(Trabalhe melhor essa ideia)
(Trata-se)
(Tratando o quê?)
(Um exemplo seria)
(Um exemplo é o que)
(Um exemplo é)
(Um)
(Usa-se "mesmo" quando se quer atribuir sentido de igualdade)
(Usa-se vírgula ao invés do ponto e vírgula)
(Use mesma quando houver sentindo de igual)
(Use pronomes relativos para evitar o queísmo no texto)
(Uso de primeira pessoa)
(Uso inadequado do ";")
(Uso obrigatório de paragráfo)
(Vale lembrar que o âmbito escolar não se resume em professores e alunos)
(Vaticano)
(Veja alguns elementos coesivos que podem melhorar a coerência no seu texto)
(Virgula)
(Vitória)
(Você citou apenas uma consequência. Cite as demais)
(Você tem esse tipo de problemas com alunos de sete anos?)
(Víirgula)
(Vírgul)
(Vírgula e letra minúscula)
(Vírgula)
(Vírgula; quais países?)
(Vítimas de quê e de quem? Deixe essa informação evidente no texto)
(Víírgula)
([,] além)

(a admissibilidade)
(a aprovação de)
(a aquisição)
(a atender)
(a ação humana)
(a corrupção)
(a data)
(a de)
(a diversidade)
(a doença)
(a escola)
(a espionagem)
(a estas)
(a evasão escolar)
(a favor deles próprios)
(a favor)
(a fim de)
(a fim)
(a frase)
(a ideia)
(a internet)
(a juventude)
(a lei da terceirização)
(a mais)
(a medida)
(a menos que queira)
(a mídia)
(a obtenção de)
(a partir)
(a pessoa)
(a publicação de)
(a que foram)
(a reforma)
(a renúncia)
(a rotina)
(a separação)
(a ser)
(a sociedade)
(a terem)
(a tv aberta)
(a ver)
(a água,)
(a – acrescentar)
(a – sem crase)
(a)
(abaixo do esperado)
(abaixo dos)
(abordagem sobre)
(abrigar)
(acaba)
(aceitação)
(acento no i)
(aconteceu)
(acredita)
(acredita-se)
(adolescentes)
(aflinge)
(agir por meio de)
(agirmos)
(agravar)
(agressiva e violeta representam a mesma ideia)
(agressão)
(agricultor )
(agricultora)
(ainda bem)
(ainda mais importante)
(ainda mais)
(ainda)
(ajudarão)
(alcunhados de “nem-nem”)
(alega estar)
(alega)
(algo)
(alguns trabalhadores)
(alguns)
(aliada à)
(almeja)
(alto)
(altos)
(alunos e professores)
(alunos)
(além da)
(além de)
(além do nordeste)
(além)
(ambiente)
(ameaçados)
(amenizar)
(americano)
(amigos virtuais)
(andar)
(aniversário)
(ano)
(anorexia e bulimia)
(anos)
(anos?)
(antes)
(antigo 2º grau)
(antítese)
(análise)
(ao invés)
(ao mesmo tempo)
(ao que se refere à quantidades ou frequência de seu uso)
(ao vínculo entre)
(ao)
(aonde estão)
(aos locais)
(aos quais)
(aos)
(apagões)
(aparentava estar)
(apelativas)
(apenas – acrescentar)
(apenas)
(apesar de que)
(aplicada atualmente)
(apoia)
(aproveitam)
(aproveitar)
(aprovou a lei que)
(aptidão natural)
(após a escrita)
(aquecimento da economia)
(argumento sem fundamento, pois o Papa João Paulo II permaneceu no pontificado até sua morte em 2 de abril de 2005)
(arma)
(artes marciais)
(artifícios)
(artigo 2)
(as Etecs )
(as atividades-fim)
(as crianças)
(as dívidas)
(as enchentes)
(as mulheres)
(as pessoas possam)
(as pessoas)
(as quais)
(as ruas)
(as)
(aspecto)
(assim como a de)
(assim como)
(assim)
(assim,)
(assinantes)
(assuntos)
(atenção, esta afirmação é falsa, certifique-se)
(atividades fim)
(atos)
(através de uma pesquisa)
(atribui-se)
(atrás)
(atualmente)
(até 6 meses)
(até a)
(até então temos Papa)
(até mesmo decente)
(até)
(atípica)
(aumentar)
(aumentou)
(autoridade mor)
(autoridades)
(autoritário)
(avaliados pelo PISA)
(avançada, não)
(avanço)
(ação ou omissão)
(aí)
(b)
(bastante reprováveis)
(bem coletivo)
(bem)
(beneficiem)
(benefício)
(bens)
(biologia)
(biomassa)
(boa parte)
(boatos da internet)
(branca e negra)
(brasileiros)
(brincadeira)
(bully = “Valentão”)
(bullying)
(bullyng)
(bônus)
(c)
(cada)
(calados!)
(caminho do guerreiro)
(caminho)
(capacidade)
(cardeais estão extremamente)
(cargo)
(carro, moto)
(casamento homossexual)
(casamento)
(casos raros)
(casos)
(castigo)
(católica)
(católicos)
(certa,)
(certos)
(chefe de estado)
(chuva)
(cicatrizar)
(cidade com)
(cidadãos brasileiros)
(cidadãos)
(circunstâncias)
(citados no primeira  )
(cite a fonte desta afirmação)
(civil e criminal)
(clerical)
(coadjuvantes)
(cobrir)
(coisas)
(colegas de classe)
(com a)
(com certeza)
(com crase)
(com essa)
(com esses fatores)
(com indignação)
(com interesses puramente econômicos)
(com o objetivo)
(com o qual)
(com o)
(com ou sem ajuda)
(com que)
(com quem vai se)
(com raras exceções)
(com ressalva para os casos de real necessidade)
(com)
(combustível)
(cometendo)
(cometer)
(cometessem)
(cometidas)
(como a retirada de um assunto que demonstre, por exemplo, a má utilização dos recursos públicos)
(como a)
(como antes acontecia)
(como compra de votos)
(como consequência)
(como consideram)
(como informado)
(como muitos alegam (Quem?)
(como muitos especulam)
(como música)
(como nas monarquias)
(como no valor da)
(como o Brasil)
(como o carro)
(como o)
(como os agrotóxicos)
(como por exemplo, decapitações de jornalistas, bombardeios e entre outros métodos)
(como por)
(como sendo)
(como sinal)
(como um resfriamento)
(como)
(como:)
(compor)
(comprometidos a)
(computador)
(comum)
(comunicação)
(comunitários)
(comuns)
(comércio)
(concedida a partir)
(conclui-se)
(confirmada)
(confuso)
(congestionamentos)
(conquistados)
(consciente)
(consciência)
(conseguiu)
(consequência)
(consequências)
(conservadorismo)
(consideradas)
(considerados sábios)
(constantemente)
(constatar)
(construir)
(contra o processo)
(contra)
(contradição)
(contrair)
(contribuem)
(contribuindo)
(contribuído)
(contrário)
(convencera)
(conveniente)
(correto)
(corrupção ativa)
(corrupção passiva)
(crase)
(crescido)
(cria)
(criança)
(crianças)
(crimes)
(cristão)
(cristãos)
(crédito)
(críticas)
(crítico)
(cumprimento)
(cão)
(d)
(da internet)
(da manhã e almoço)
(da população)
(da renúncia)
(da)
(dada)
(dali)
(daqueles)
(das vezes)
(das)
(dava)
(daí o seu nome)
(de 25 para 21 anos)
(de 5 a 10 gramas)
(de Papa)
(de acordo com a)
(de acordo com)
(de armas)
(de chuvas)
(de drogas)
(de esportes)
(de face á face)
(de fiéis)
(de forma destacada(,)
(de forma moderada)
(de forma significativa)
(de freiar o ciclo)
(de idade)
(de iniciativa popular)
(de maioria negra)
(de modo que)
(de privacidade)
(de qualquer gênero)
(de qualquer religião)
(de que na)
(de que)
(de se)
(de ser)
(de suas)
(de trabalho)
(de um modo)
(de voto)
(de)
(debilidade e cansaço)
(debilidade física)
(debilidade)
(decapitado)
(decisão)
(declarar)
(declarações)
(declínio)
(decorrentes de)
(decpitando pessoas)
(decreta que)
(decreto dos plebeus)
(defende)
(defendendo a ideia)
(define que)
(deixa)
(deixando)
(deixaram)
(deixou)
(demandam da pessoa)
(demo)
(democracia)
(denúncias)
(dependendo da criação)
(depois de)
(desconfiança)
(desde que haja)
(desde que)
(desde)
(desenvolver)
(desespero)
(desidratados)
(desigualdade racial)
(desigualdade)
(despertar)
(despertou)
(despreparados)
(dessa)
(dessas substâncias)
(desta gravidade)
(desvínculo)
(determina a)
(deve aparecer entre aspas, por se tratar de ironia)
(deve haver um)
(deve haver)
(deve se)
(deve ser)
(deve ter)
(deve vir)
(deve)
(deve-se)
(devem ser firmadas)
(devem ser realizados)
(devem ser)
(devem)
(devemos)
(deveriam)
(devido)
(devotos, ou não)
(dia a dia)
(diante do)
(dias extracurriculares)
(didáticos)
(diesel e gasolina)
(dificuldade na)
(dificíl)
(dinheiro este que)
(dinheiro)
(dinheiro, patrimônio )
(direito a)
(direito)
(direitos civis)
(direitos e deveres)
(direitos)
(discussão)
(discussões)
(discuti-lo)
(discutidos pela)
(disposição)
(disque100)
(disseram)
(disso)
(disso, policiais são)
(disso, é importante)
(distinguir)
(distribuição)
(distritos)
(ditadura)
(diversos)
(dividindo)
(divulgado pela)
(divulgados)
(diz)
(dizer o mínimo)
(diálogo)
(dióxido de carbono)
(do Governo Federal)
(do fato de)
(do machismo)
(do mesmo)
(do que é)
(do vírus)
(do)
(doações e propaganda)
(dois mil e treze)
(dois pontos)
(domínio)
(dos)
(drogas, assaltos)
(durante o)
(durante)
(dívidas)
(dúvida)
(e / retire a vírgula)
(e a criação de)
(e a)
(e acredita nelas)
(e afins)
(e afirmava que)
(e ainda há)
(e as)
(e até trágico)
(e com quem)
(e condenados)
(e cuja)
(e desejável)
(e do)
(e dos adolescentes)
(e especialmente)
(e financiado)
(e há rumores)
(e levá-las para)
(e mataram)
(e muito!)
(e muito)
(e o Executivo)
(e o catolicismo)
(e o)
(e onde)
(e pessoas)
(e por)
(e possível)
(e representam)
(e seus representantes)
(e sim)
(e sinônimo)
(e sites)
(e um problema)
(e votos)
(e é)
(e única até então)
(e)
(e,)
(efeito dominó)
(ela)
(elaboradas)
(elaborá-las)
(elas)
(ele é)
(eles mesmos)
(eles)
(em Bangladesh)
(em Nova York)
(em conta que)
(em destaque)
(em discussão)
(em educação)
(em episódios)
(em falta de água)
(em frente)
(em geral)
(em jornais)
(em julho de 2014)
(em prol)
(em quaisquer que seja)
(em que)
(em reais)
(em relação a)
(em relação à)
(em relação)
(em tempo)
(em uma)
(em vigor)
(em vista o)
(em)
(embora pouco didática)
(empregadores)
(empregados/sindicatos)
(empresário)
(en)
(encontra-se)
(encontram)
(energia eólica)
(energia geotérmica)
(energia maremotriz)
(energia solar)
(energica elétrica)
(enfim)
(ensimesmar-se)
(entre aspas – “liberalização”)
(entre aspas)
(entre elas o consumismo)
(entre)
(equívoco)
(era)
(erroneamente)
(es)
(escolar)
(escolaridade)
(escravos)
(escritor solitário)
(espaço e professores)
(espaço)
(espaços)
(especialmente a elite)
(especialmente o)
(específica)
(espionagem virtual)
(esposo (a)
(esquerda e direita)
(esquerda ou direita)
(essa juventude)
(essa liberdade)
(essa quantidade de água)
(essas atitudes)
(essas)
(esse)
(esses sim culpados)
(esses)
(esta)
(estado)
(estar)
(estarem em evidência)
(estarem)
(estava)
(este dispositivo)
(este)
(estes)
(estima-se 20 milhões)
(está relacionada)
(está)
(estão)
(etc [etc.])
(etc)
(etc.)
(eticamente)
(etnias)
(etnocentrismo)
(eventuais)
(evidenciam-se)
(evidência)
(evitados)
(evitem a)
(evolução)
(evoluírem)
(ex)
(ex. poliomielite)
(examinar)
(executivo)
(exemplares do)
(exemplo)
(exista)
(existe)
(existem exceções)
(existem)
(experiências)
(externa)
(extinção)
(facebook, twiter )
(fair play)
(falta de)
(família)
(fator)
(fatores)
(faz o)
(faz-se)
(fazendo-o)
(façam)
(feita)
(feito)
(felicidade)
(fervoroso)
(fetichismo)
(feto)
(fetos de)
(fevereiro)
(fica oculto)
(fica)
(ficam)
(ficar somente)
(ficha)
(fictícios)
(filhos)
(fim da corrupção)
(final do ano)
(fiéis)
(floresciam)
(foi feito)
(foi no)
(foi)
(folha de são paulo)
(fonte)
(for)
(fora do)
(foram chamados)
(foram)
(forem)
(forma)
(fosse)
(fossem)
(fotos,vídeos,música)
(fraca e instável)
(frente a)
(frustrada)
(fundador da Vitacon)
(funk)
(funções típicas)
(fácil)
(física e moral)
(física)
(g1)
(gasolina)
(gasto)
(gera)
(geram)
(governo)
(grampeamentos)
(grandes pecuriaristas)
(graves)
(gravidez)
(grupo terrorista)
(grupos de)
(grupos)
(guerra santa)
(guerras santas)
(h)
(hajam)
(havia dito)
(hegemonia)
(heresias)
(hidrelétricas)
(hipercapitalismo)
(homens e mulheres)
(homossexuais)
(homossexualidade)
(horária)
(horários)
(hostil)
(houve)
(houveram)
(há a)
(há exceções, é claro!)
(há)
(hábito)
(ideia confusa.Revise)
(ideias)
(identidade falsa)
(ilícita)
(imagem)
(impacto)
(imparciais)
(impor)
(importância)
(impunes)
(in)
(incentivo)
(incentivá-las)
(inclusive a imprensa)
(inclusive ilícitas)
(inclusive)
(inclusão social)
(incomodar)
(indigna)
(individuais)
(indivíduo)
(indivíduos)
(inegável)
(infantis)
(infelizmente)
(influência)
(informada)
(informação dispensável)
(informação)
(infrações)
(infringir)
(infringirem)
(inicie parágrafos com letra maiúscula)
(inserção)
(instituição católica)
(instituições públicas)
(intenção de matar)
(interagem)
(internet)
(interno)
(intolerável)
(intoxicação e morte)
(intrigam)
(investimentos)
(início da República)
(início de período com letra maiúscula)
(início do)
(início)
(ir)
(ira)
(irmão ou filho)
(irrestrito)
(irá)
(irão)
(isso)
(isto é(,)
(jovens)
(judiciário)
(juntos)
(já garantidos)
(já não)
(já)
(kracia)
(kratos)
(lado)
(ladrões)
(lavagem)
(legislativo)
(lei da)
(letra maiúscula)
(letra minúscula)
(levada em consideração)
(levados)
(levam)
(levando à)
(levando)
(levar)
(levou)
(liberados)
(linear ?)
(longe de acabar)
(longe de áreas de risco)
(lésbicas e travestis)
(lê-se coligações)
(líder)
(líderes)
(m)
(mTa)
(magras)
(maior)
(maiores)
(maioria armados)
(maioridade)
(mais populares)
(mais se)
(mais tarde)
(mais violentas)
(mais)
(maiúscula)
(mal)
(manda)
(manifestava-se)
(manifestações)
(marca)
(marco)
(março)
(mas há)
(mas para outros)
(mas sim)
(mas)
(matéria)
(mau)
(maíusculo)
(medidas provisórias)
(medidor da desigualdade)
(medievais)
(meio de)
(melhora no)
(melhora)
(melhore esse argumento)
(menores de 18 anos)
(menos o)
(menos)
(mesmo inconscientemente)
(mesmo que)
(mesmo sem autorização)
(mesmo)
(mesmos)
(microcefalia)
(mil)
(ministro da Educação,)
(ministro da fazenda)
(minúscula)
(minúsculo)
(mix martial arts)
(momentâneos)
(monopólio)
(morte)
(mostraram)
(motivo)
(motivos)
(movimentos sociais)
(mudança na)
(mude)
(muitas)
(muito trabalho)
(muito)
(muitos)
(multas)
(mundiais)
(município)
(mutantes)
(m³)
(más)
(máximo)
(mãos)
(média)
(médicos)
(mês)
(mídia)
(mínima)
(n)
(na China)
(na atualidade)
(na classe mais alta)
(na disciplina)
(na escolha do povo )
(na maioria dos casos)
(na prática)
(na qual)
(na verdade, por séculos)
(na Índia)
(na área)
(na)
(nacionais de pelo menos)
(nada)
(nas cidades)
(nas mortes)
(nas redes sociais)
(natural de Alagoas)
(ndo)
(necessário)
(negativo)
(negros e indígenas)
(nenhuma)
(nesse caso não use a vírgula)
(neste caso não é necessário o uso da vírgula)
(neste caso,não pe necessário o uso sa vírgula)
(neste caso,não é necessário o uso de vírgula)
(netfobia)
(ninguém)
(no Brasil)
(no México)
(no aspecto de que)
(no caso quem pagar mais)
(no caso(,)
(no fato de)
(no fornecimento)
(no mínimo)
(no período colonial)
(no século XX)
(no)
(nordestinos)
(normais)
(norte e leste)
(nos)
(nosso)
(notícia)
(notória)
(novamente)
(nove)
(noção)
(nte)
(nunca)
(não aceita?)
(não agredir)
(não apenas políticos)
(não apenas)
(não deve estar entre aspas)
(não entendi o que quis dizer)
(não há virgula)
(não há vírgula)
(não inclusão na CLT)
(não me arrependo)
(não passiva!)
(não podem se fundir)
(não se usa vírgula neste caso)
(não se utiliza vírgula neste caso)
(não são mais uma opção,)
(não tem crase)
(não tirar vermelho)
(não é necessário o uso de vírgula)
(não)
(não-fumantes)
(nível médio)
(nós)
(número do candidato)
(número)
(o "caipira")
(o "eu")
(o "nerd")
(o "nós")
(o "orelhudo")
(o Brasil em)
(o Brasil)
(o Estado)
(o aluno)
(o assunto mais tratado)
(o autor)
(o avanço)
(o caso)
(o chamado plano B)
(o consumo)
(o contrário)
(o crussifixo)
(o de)
(o direito)
(o fato)
(o gado)
(o golpe republicano)
(o governo federal)
(o governo)
(o homem)
(o ministro)
(o núcleo na história)
(o número de automóveis)
(o número de)
(o país)
(o problema)
(o próprio)
(o qual)
(o que afeta)
(o que causa)
(o que possilibita)
(o que sempre aconteceu)
(o que é errado)
(o que)
(o relacionamento)
(o religioso)
(o sistema Cantareira)
(o tema)
(o uso do preservativo)
(o virtual)
(o vírus zika)
(o índice)
(o)
(obrigatório)
(obtêm)
(ocorre)
(ocorrem inúmeros)
(ocorrido)
(ofereceu)
(oito por cento)
(olharmos)
(onde a maioria é negra)
(onde só o rico tem vez)
(onde)
(opiniões)
(opções)
(organização interna)
(orientação sexual)
(origem e destino)
(origem)
(originalmente)
(os alunos)
(os anos)
(os brasileiros)
(os filósofos)
(os membros da)
(os membros do clero)
(os menos favorecidos)
(os pobres!)
(os poderes)
(os políticos)
(os quais rompem)
(os quais)
(os shoppings)
(os terroristas)
(os)
(ou a desgraça)
(ou acomodaram-se)
(ou aculturação)
(ou até mais)
(ou até nenhum)
(ou até pior)
(ou de querer)
(ou falta)
(ou favelas)
(ou finge não perceber)
(ou frontal)
(ou homoafetivos)
(ou inexistente)
(ou invadido)
(ou libera poucos)
(ou mais)
(ou mal)
(ou mamãe)
(ou marido )
(ou nem isso)
(ou não!)
(ou não)
(ou por)
(ou possui descendência)
(ou projetos de homens)
(ou pseudointelectuais)
(ou quase todos)
(ou se lembra vagamente)
(ou seja, os mais ricos)
(ou sempre)
(ou sendo inseridas)
(ou ser alvo dela!)
(ou simpatizantes)
(ou talvez mocinho)
(ou toxinas bacterianas)
(ou)
(outras)
(outro pontífice,)
(outro)
(outros)
(padres)
(pague)
(pais)
(palavra excedente)
(palestinos)
(papado)
(papéis)
(para 16 anos)
(para a formação de uma)
(para a nova escolha)
(para ambas as partes)
(para eles)
(para o)
(para os adultos)
(para que haja)
(para que não o faça)
(para que)
(para todos)
(para)
(parte dos)
(parágrafo sem fundamento)
(passaram)
(passe por)
(país desenvolvido)
(país que)
(país)
(países como a)
(países)
(pecado contra Deus)
(pedofilia)
(pedófilos)
(pegos)
(peixes e aves)
(pel)
(pela)
(pelas quais)
(pelo menos um)
(pelo)
(pelos cidadãos)
(pelos)
(penas mais severas a)
(penas)
(penitenciárias)
(pensar)
(perceberemos que)
(perda)
(perdeu parte da orelha)
(pergunta-se)
(período confuso – a criminalidade é um deles, “um deles” o que? Quem são eles?)
(período confuso)
(período confuso, articule melhor)
(períodos de)
(pesquisa do)
(pessoa)
(pessoais)
(pessoas)
(petróleo)
(peças)
(pobres e negros)
(pode fazer com que)
(pode torná-la finita)
(pode)
(podem tomar)
(podem)
(poder)
(poderia ser)
(poderia)
(poderão)
(pois colabora com)
(pois família)
(pois)
(poluição das águas)
(polêmica)
(polêmicas)
(polêmico)
(política é)
(política)
(políticas e jurídicas)
(políticas)
(político)
(políticos e sociais)
(políticos)
(ponto final e início de período com letra maiúscula)
(ponto final)
(por Deus)
(por ambos os sexos)
(por cento)
(por eles)
(por exemplo)
(por meio da mídia)
(por meio de)
(por meio do)
(por meio)
(por parte dos)
(por problemas)
(por que)
(por saber)
(por terem cometidos)
(por)
(porque)
(portanto sub-raça)
(porém é)
(porém)
(possa)
(possui)
(possuíam)
(possíveis)
(possível por meio de)
(possível)
(poém)
(praticados)
(praticam)
(praticante de bullying)
(praticar)
(praticou o)
(praça pública)
(praças públicas gregas)
(preciso)
(predadores naturais)
(prendem)
(preocupado)
(preocuparem)
(presidenta do Brasil)
(presidente dos EUA)
(presidente)
(prestam)
(presídio)
(presídios)
(primeira pessoa do plural)
(primeiro Papa)
(primeiros)
(principalmente jovens)
(principalmente)
(privado)
(privados)
(problema)
(problemas no)
(problemas)
(procure não utilizar essa expressão; utilize “Atualmente”)
(progresso econômico)
(proibir ou apoiar?)
(projeto de)
(promessa)
(pronunciamentos)
(proposital)
(prostituição)
(provavelmente)
(proveniente do metano)
(providências)
(provocaram)
(prática)
(práticas sociais)
(pré)
(própria)
(próprias)
(próximo de rios)
(próximos)
(psicológica)
(psicológico)
(publicado)
(punido)
(punidos)
(pão e circo)
(pós-guerra)
(pública)
(públicas e privadas)
(públicas)
(públicos ou privados)
(quais as fontes que comprovam isso?)
(quais filosofos? Cite pelo um para comprovar seus argumentos)
(quais)
(quando bem cuidados)
(quando ocorre)
(quanto de)
(quanto para)
(quanto)
(quarenta)
(quase sempre)
(quase)
(que acabou de comprar)
(que deve haver)
(que deve ser)
(que está lá fora)
(que fizeram com)
(que foi manipulada)
(que foi)
(que haja)
(que há)
(que já não é pequena)
(que nunca teve fim)
(que não mentem)
(que não é dele)
(que o)
(que ocorreu)
(que os espera)
(que poderiam ser(,)
(que querem lucrar)
(que se faça)
(que se sucede)
(que se trata de uma)
(que são boas)
(que são)
(que também é um dever)
(que ter)
(que têm o)
(que é)
(que)
(que, em vez de)
(queda)
(quem)
(quer dizer(,)
(querem que)
(quinhentos mil)
(quinto)
(quis)
(quiserem)
(quão)
(r)
(rainhas do Egito)
(ranking)
(re)
(real problema social)
(real)
(realidade social)
(realizadas)
(realizado em 2010)
(realizar)
(receber)
(recentemente)
(rede privada)
(rede pública)
(redes sociais)
(redes)
(redudância)
(redundância)
(referente)
(reforma)
(regras)
(relaciona-se)
(relacionadas à miséria)
(relacionadas)
(relação a)
(relação à)
(relevantes)
(religiosa)
(religião católica)
(remuneração)
(renuncia a seu)
(renuncia a)
(renuncia ao cargo)
(renuncia)
(renunciar)
(renunciaram)
(renúncia repentina)
(renúncia)
(renúncia,)
(representando a gestão)
(representantes do povo)
(representá-los)
(repórter)
(responda à essa pergunta no máximo no inpicio do proximo parágrafo)
(responsáveis)
(responsável)
(ressaltaram)
(retirou)
(retornando)
(revela que)
(revertidos)
(revisar)
(ricos)
(risco de ser morto)
(rodovias)
(roubo seguido de morte)
(rua)
(ruas das)
(ruas pelas)
(rádio, cinema, TV)
(rúgbi)
(s)
(sa)
(sabemos que é difícil)
(saem)
(sala de aula)
(salvo exceções)
(salário mínimo)
(salários)
(sancionado em 1940)
(saíram)
(saúde)
(se alaga)
(se houver)
(se mudaram)
(se não todos)
(se passaram)
(se tornasse)
(se é que eles existem!)
(se é que tem!)
(se)
(segue)
(seguidores)
(segundo pesquisas)
(seja)
(sem "se")
(sem acento)
(sem assunto)
(sem atração)
(sem crase)
(sem limite)
(sem o "e")
(sem um)
(sem vírgula)
(sem vírgula, acrescentar o ponto final)
(sem vírgula, use o ponto final)
(sem)
(semanário)
(sempre lotados)
(sendo assim)
(sendo assim,)
(sendo que)
(sendo)
(senhor feudal)
(ser aplicadas)
(ser colocado)
(ser realizada)
(ser)
(serem)
(seria que)
(seria)
(serve)
(serviços indevidos)
(será a reação)
(será necessário tomar )
(será possível que)
(será)
(serão reduzidas)
(serão vencedores.)
(serão)
(sete)
(seu filho)
(seu marido)
(seu)
(seus alunos)
(seus candidatos)
(seus)
(severa)
(shopping~~s~~ centers)
(sido)
(silêncio)
(sim)
(singular)
(sistema que)
(sistema único de saúde)
(sistema)
(situação)
(situações)
(sobre qual)
(sobre)
(social e governamental)
(socialmente)
(sociedade e Estado)
(sociedade linear)
(socioeducativas)
(socioeducativos)
(somente)
(somos)
(sozinhos)
(startup)
(sua eleição)
(sua maioria)
(sua)
(suas)
(subliminarmente)
(suficientes)
(sugere)
(superior a)
(surgirá)
(surpreendeu)
(sustentabilidade)
(são cada vez mais)
(são incentivadas)
(são necessários)
(são)
(século XIX)
(século XVII)
(série)
(símbolo cristão)
(símbolos de respeito)
(só que em larga escala)
(só)
(súmulas vinculantes)
(tal como já está sendo)
(tal fato)
(tal qual a Aids)
(tal)
(talvez)
(também na)
(também têm)
(também)
(tanto)
(tanto,quanto)
(tantos)
(tem)
(tema)
(temendo)
(temos como)
(temos)
(tenha sido)
(tenha)
(teologicamente)
(teoricamente)
(ter acobertado)
(ter)
(terceira dimensão)
(terem)
(teria)
(termo entre aspas)
(termo esdrujulo/ em vão)
(termos)
(terão)
(tese)
(teve início)
(texto anônimo)
(the slutwalk)
(tido)
(tinham)
(tipo)
(todo)
(todos os)
(tomar)
(tome cuidado ao utilizar esta palavra, certifique-se primeiro)
(torna-se)
(tornassem)
(tornem-se)
(trabalhista, familiar)
(tradicional)
(traficante)
(traficantes)
(tragédia?)
(trará)
(tratando)
(traz)
(trazê-la)
(trecho incoerente)
(tributária, agrária)
(trouxesse)
(tráfico)
(três anos)
(três)
(têm buscado)
(têm dificuldades de interpretação e de expressão)
(têm)
(típica)
(título)
(um empecilho)
(um número suportável)
(um palhaço!)
(um total de)
(um)
(uma arma)
(uma edição)
(uma parcela)
(uma pré-copa do mundo)
(uma)
(unidades)
(universais)
(usa-se "mais", pois é dada a ideia de adição)
(use a 3ª pessoa)
(use a vírgula)
(use letra maíscula para se referir a determinada Instituição anteriormente citada)
(uso da vírgula)
(uso de letra maiúscula)
(uso de letra minúscula)
(uso de ponto final)
(uso de vígula)
(uso de vírgula)
(uso de)
(uso desnecessário)
(utilidade)
(utilize a 3ª pessoa)
(utilize outro termo)
(vale)
(valentão)
(veio a falecer)
(vem)
(vento, sol...)
(verem)
(vestibular)
(vez)
(vieram)
(vinte anos)
(vinte três por cento)
(vinte)
(violentos)
(visando ser maioria)
(visão e audição)
(visíveis)
(visível)
(vive)
(vivência)
(você já caracterizou no primeiro paragrágo que é no Brasil)
(voltam)
(voltou à tona)
(várias polêmicas)
(várias)
(vários países)
(vê)
(vê-se)
(vêm)
(vírgula)
(vítima)
(vítimas)
(~~ no PISA~~)
(~~;~~)
(~~Cid Gomes~~)
(~~com~~)
(~~e~~ maioria)
(~~ibeb~~ ideb)
(~~ruas (com)
(África do Sul)
(É interessante mencionar dados para dar validade a essa argumentação)
(É preciso apresentar mais argumentos para que isso fique evidente)
(É)
(Édito)
(à Igreja Católica)
(à base de duras lutas)
(à esta)
(à nação e a corrompend)
(à pedofilia)
(à – nesse caso use a crase)
(à – use a crase)
(à – uso da crase)
(à – uso de crase)
(à – utilize a crase)
(à)
(às - uso de crase)
(às)
(á anos atrás)
(á)
(água doce)
(águas subterrâneas)
(área)
(áreas)
(ão)
(é a)
(é algo que)
(é importante que)
(é importante saber)
(é imprescindível)
(é necessário)
(é o mais duradouro)
(é possível)
(é preciso se)
(é preciso)
(é que são)
(é que,)
(é um aprendizado que)
(é um)
(é)
(étnicos)
(ê)
(índice)
(ó)
(ótimos)
(última)
(últimas denúncias)
(últimas)
(último)
(últimos)
(única)""".split('\n')

